In [2]:
import torch
print(f'CUDA available: {torch.cuda.is_available()}')
print(f'GPU: {torch.cuda.get_device_name(0)}')
print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

from google.colab import drive
drive.mount('/content/drive')


# Create a persistent project directory
import os
PROJECT_DIR = '/content/drive/MyDrive/AI574_Multi_Domain_Agent'
os.makedirs(PROJECT_DIR, exist_ok=True)
os.makedirs(f'{PROJECT_DIR}/data', exist_ok=True)
os.makedirs(f'{PROJECT_DIR}/models', exist_ok=True)
print(f'Project directory: {PROJECT_DIR}')


CUDA available: True
GPU: NVIDIA A100-SXM4-80GB
Memory: 85.1 GB
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Project directory: /content/drive/MyDrive/AI574_Multi_Domain_Agent


In [3]:
# Action: Run this cell to install all packages:

# %%capture
!pip install -q \
  "langchain>=0.3.0" "langchain-core>=0.3.0" "langgraph>=0.2.0" "langchain-text-splitters>=0.2.0" \
  "keras>=3.0" "keras-hub>=0.17.0" \
  "sentence-transformers>=3.0.0" "transformers>=4.45,<5.0" "chromadb>=0.5.0" \
  "PyPDF2>=3.0.0" "arxiv>=2.1.0" "pydantic>=2.0"

!pip install -q "jax[cuda12]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 116.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 133.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.4

In [4]:
import os
os.environ['KERAS_BACKEND'] = 'jax'

# Verify
import keras
print(f'Keras backend: {keras.backend.backend()}')
print(f'Keras version: {keras.__version__}')

Keras backend: jax
Keras version: 3.10.0


In [5]:
import langchain
import langgraph
import keras_hub
import chromadb
from sentence_transformers import SentenceTransformer
import jax
print('All imports successful!')
print(f'JAX devices: {jax.devices()}')

All imports successful!
JAX devices: [CudaDevice(id=0)]


In [6]:
import os
import sys

# PROJECT_DIR is already defined
PROJECT_ROOT = os.path.join(PROJECT_DIR, 'project')

dirs = [
    'config',
    'foundation',
    'ingestion',
    'rag_core',
    'agents',
    'orchestration',
    'evaluation',
    'data/industrial',
    'data/recipes',
]

# Create directories
for d in dirs:
    os.makedirs(os.path.join(PROJECT_ROOT, d), exist_ok=True)

# Create __init__.py in every package directory
packages = [
    'config',
    'foundation',
    'ingestion',
    'rag_core',
    'agents',
    'orchestration',
    'evaluation'
]

for pkg in packages:
    init_file = os.path.join(PROJECT_ROOT, pkg, '__init__.py')
    open(init_file, 'a').close()

# Add project root to Python path
sys.path.insert(0, PROJECT_ROOT)

print('Project structure created at:', PROJECT_ROOT)

Project structure created at: /content/drive/MyDrive/AI574_Multi_Domain_Agent/project


In [7]:
import os, sys

# PROJECT_DIR already defined elsewhere
PROJECT_ROOT = os.path.join(PROJECT_DIR, "project")
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("PROJECT_ROOT =", PROJECT_ROOT)
print("foundation exists?", os.path.isdir(os.path.join(PROJECT_ROOT, "foundation")))
print("init exists?", os.path.isfile(os.path.join(PROJECT_ROOT, "foundation", "__init__.py")))
print("files:", os.listdir(os.path.join(PROJECT_ROOT, "foundation")))

PROJECT_ROOT = /content/drive/MyDrive/AI574_Multi_Domain_Agent/project
foundation exists? True
init exists? True
files: ['embedding_service.py', 'vector_store.py', '__init__.py', '__pycache__', 'llm_wrapper.py']


In [13]:
# ── Sync: write local source files to PROJECT_ROOT on Google Drive ────────────
# This cell embeds all .py files from your local project.
# Run it to write them to Drive so Colab uses the latest code.
# To update: edit files locally in Cursor, then re-run the sync script.

import os, sys

_FILES = {

    'agents/__init__.py': r'''
''',

    'agents/base_agent.py': r'''
"""
Base Agent
==========
Abstract base class for all specialized agents.
Each agent = domain config + optional preprocessing + shared CRAG pipeline.

Subclasses only need to override:
- domain (str)
- preprocess_query() — optional domain-specific query prep
- postprocess_response() — optional domain-specific formatting
"""

from __future__ import annotations

import logging
from abc import ABC, abstractmethod
from typing import Optional

from rag_core.crag_pipeline import CRAGPipeline, CRAGResult

logger = logging.getLogger(__name__)


class BaseAgent(ABC):
    """Abstract base for domain-specialized agents."""

    def __init__(self, crag_pipeline: CRAGPipeline):
        self.crag = crag_pipeline

    @property
    @abstractmethod
    def domain(self) -> str:
        """Domain identifier (matches vector store collection key)."""
        ...

    @property
    @abstractmethod
    def description(self) -> str:
        """Human-readable description of this agent's capabilities."""
        ...

    def handle(self, query: str, **kwargs) -> CRAGResult:
        """
        Full agent execution: preprocess → CRAG → postprocess.

        This is the main entry point called by the supervisor.
        """
        logger.info(f"[{self.domain}] Handling query: {query[:80]}...")

        # Domain-specific query preprocessing
        processed_query = self.preprocess_query(query)

        # Run shared CRAG pipeline
        result = self.crag.run(
            query=processed_query,
            domain=self.domain,
            **kwargs,
        )

        # Domain-specific postprocessing
        result = self.postprocess_response(result)

        return result

    def preprocess_query(self, query: str) -> str:
        """
        Optional domain-specific query preprocessing.
        Override in subclass to add terminology expansion, etc.
        Default: pass-through.
        """
        return query

    def postprocess_response(self, result: CRAGResult) -> CRAGResult:
        """
        Optional domain-specific response postprocessing.
        Override in subclass to format output, add warnings, etc.
        Default: pass-through.
        """
        return result
''',

    'agents/industrial_agent.py': r'''
"""
Industrial Troubleshooting Agent
================================
Specialized for PLC/SCADA diagnostics, fault code interpretation,
and maintenance procedures. Adds industrial-specific query preprocessing
and safety warnings to responses.
"""

from __future__ import annotations

import logging
import re

from agents.base_agent import BaseAgent
from rag_core.crag_pipeline import CRAGPipeline, CRAGResult

logger = logging.getLogger(__name__)

# Known fault code patterns by vendor
FAULT_CODE_PATTERNS = {
    "siemens": re.compile(r'\b[Ff]\d{4,5}\b'),           # F0003, F07900
    "allen_bradley": re.compile(r'\bF\d{1,3}\b'),         # F2, F33
    "generic": re.compile(r'\b(?:fault|error|alarm)\s*(?:code\s*)?[:#]?\s*\d+', re.I),
}

# Equipment model patterns
EQUIPMENT_PATTERNS = re.compile(
    r'\b(S7-\d{3,4}|1[257]\d{2}|6ES\d|PanelView|PowerFlex\s*\d+)\b', re.I
)


class IndustrialAgent(BaseAgent):
    """
    Industrial Equipment Troubleshooting Specialist.

    Enhancements over base agent:
    - Extracts and normalizes fault codes from queries
    - Expands industrial abbreviations
    - Adds safety warnings to responses involving electrical/mechanical work
    """

    @property
    def domain(self) -> str:
        return "industrial"

    @property
    def description(self) -> str:
        return (
            "Industrial equipment troubleshooting specialist. Handles PLC/SCADA "
            "diagnostics, fault code interpretation, motor drives, industrial "
            "networking, and maintenance procedures."
        )

    def preprocess_query(self, query: str) -> str:
        """
        Industrial-specific query preprocessing:
        1. Extract and normalize fault codes
        2. Identify equipment models
        3. Expand common abbreviations for better retrieval
        """
        processed = query

        # Extract fault codes and make them explicit
        for vendor, pattern in FAULT_CODE_PATTERNS.items():
            matches = pattern.findall(processed)
            if matches:
                # Ensure fault code is prominent for retrieval
                codes = ", ".join(matches)
                if "fault" not in processed.lower():
                    processed += f" (fault code: {codes})"
                logger.debug(f"Extracted fault codes ({vendor}): {codes}")

        # Extract equipment models
        equip_matches = EQUIPMENT_PATTERNS.findall(processed)
        if equip_matches:
            logger.debug(f"Detected equipment: {equip_matches}")

        return processed

    def postprocess_response(self, result: CRAGResult) -> CRAGResult:
        """
        Add safety warnings for responses involving potentially
        dangerous procedures (electrical, mechanical, pressurized systems).
        """
        safety_keywords = [
            "high voltage", "lockout", "tagout", "loto",
            "energized", "live wire", "arc flash",
            "pressurized", "hydraulic", "pneumatic",
            "rotating", "pinch point", "confined space",
        ]

        response_lower = result.response.lower()
        triggered_warnings = [
            kw for kw in safety_keywords if kw in response_lower
        ]

        if triggered_warnings:
            safety_notice = (
                "\n\n⚠️ **SAFETY WARNING**: This procedure may involve "
                "hazardous conditions. Ensure all applicable safety protocols "
                "(lockout/tagout, PPE, permits) are followed. Consult your "
                "site safety officer before proceeding."
            )
            result.response += safety_notice
            logger.info(f"Safety warning added (triggers: {triggered_warnings})")

        return result
''',

    'agents/recipe_agent.py': r'''
"""
Recipe Assistant Agent
======================
Specialized for cooking guidance, ingredient substitutions,
and recipe recommendations. Adds ingredient-aware query preprocessing
and nutrition highlighting.
"""

from __future__ import annotations

import logging
import re

from agents.base_agent import BaseAgent
from rag_core.crag_pipeline import CRAGPipeline, CRAGResult

logger = logging.getLogger(__name__)

# Common substitution triggers
SUBSTITUTION_KEYWORDS = [
    "substitute", "replace", "instead of", "alternative",
    "swap", "without", "allergy", "intolerant", "vegan",
    "dairy-free", "gluten-free", "no eggs",
]

# Cooking technique synonyms for query expansion
TECHNIQUE_SYNONYMS = {
    "fry": ["sauté", "pan-fry", "deep-fry", "stir-fry"],
    "bake": ["roast", "oven"],
    "boil": ["simmer", "poach", "blanch"],
    "grill": ["broil", "char-grill", "barbecue"],
}


class RecipeAgent(BaseAgent):
    """
    Recipe Assistant Specialist.

    Enhancements over base agent:
    - Detects substitution queries and adds context
    - Expands cooking technique terms for broader retrieval
    - Tags dietary information in responses
    """

    @property
    def domain(self) -> str:
        return "recipe"

    @property
    def description(self) -> str:
        return (
            "Recipe assistant for cooking guidance, ingredient substitutions, "
            "recipe search, nutritional information, and food preparation techniques."
        )

    def preprocess_query(self, query: str) -> str:
        """
        Recipe-specific query preprocessing:
        1. Detect substitution intent and enrich query
        2. Expand cooking technique terms
        """
        processed = query
        query_lower = query.lower()

        # Check for substitution queries
        is_substitution = any(kw in query_lower for kw in SUBSTITUTION_KEYWORDS)
        if is_substitution:
            # Add "substitution" explicitly for retrieval boost
            if "substitut" not in query_lower:
                processed += " substitution alternative"
            logger.debug("Detected substitution query")

        # Expand cooking techniques
        for technique, synonyms in TECHNIQUE_SYNONYMS.items():
            if technique in query_lower:
                expansion = " ".join(synonyms[:2])  # Add top 2 synonyms
                processed += f" {expansion}"
                logger.debug(f"Expanded technique '{technique}' with: {expansion}")
                break

        return processed

    def postprocess_response(self, result: CRAGResult) -> CRAGResult:
        """
        Add dietary tags if relevant dietary terms are detected
        in the response.
        """
        dietary_tags = {
            "vegan": ["vegan", "plant-based"],
            "vegetarian": ["vegetarian", "no meat"],
            "gluten-free": ["gluten-free", "celiac"],
            "dairy-free": ["dairy-free", "lactose"],
            "nut-free": ["nut-free", "nut allergy"],
        }

        response_lower = result.response.lower()
        detected_tags = [
            tag for tag, keywords in dietary_tags.items()
            if any(kw in response_lower for kw in keywords)
        ]

        if detected_tags:
            tags_str = " | ".join(f"🏷️ {tag}" for tag in detected_tags)
            result.response += f"\n\n{tags_str}"

        return result
''',

    'agents/scientific_agent.py': r'''
"""
Scientific Paper Summarizer Agent
==================================
Specialized for academic literature synthesis via ArXiv.
Supports on-demand paper retrieval and citation-aware summarization.
"""

from __future__ import annotations

import logging
from typing import Optional

from agents.base_agent import BaseAgent
from rag_core.crag_pipeline import CRAGPipeline, CRAGResult
from ingestion.index_builder import IndexBuilder

logger = logging.getLogger(__name__)


class ScientificAgent(BaseAgent):
    """
    Scientific Paper Summarization Specialist.

    Enhancements over base agent:
    - On-demand ArXiv fetching when local index is insufficient
    - Adds citation formatting to responses
    - Structures summaries as Objective → Method → Findings → Limitations
    """

    def __init__(
        self,
        crag_pipeline: CRAGPipeline,
        index_builder: Optional[IndexBuilder] = None,
    ):
        super().__init__(crag_pipeline)
        self.index_builder = index_builder

    @property
    def domain(self) -> str:
        return "scientific"

    @property
    def description(self) -> str:
        return (
            "Scientific paper summarization specialist. Handles ArXiv research "
            "queries, literature synthesis, and academic concept explanation."
        )

    def handle(self, query: str, **kwargs) -> CRAGResult:
        """
        Extended handle: if local retrieval fails AND we have an index builder,
        fetch fresh papers from ArXiv and retry.
        """
        # First, try with existing indexed papers
        result = super().handle(query, **kwargs)

        # If escalated due to no relevant docs, try fetching from ArXiv
        if (
            result.escalated
            and self.index_builder
            and "No documents" in result.escalation_reason
        ):
            logger.info("Local retrieval failed — fetching from ArXiv on-demand")
            try:
                count = self.index_builder.index_arxiv_papers(
                    query, max_results=10
                )
                if count > 0:
                    logger.info(f"Indexed {count} new ArXiv papers, retrying...")
                    result = super().handle(query, **kwargs)
            except Exception as e:
                logger.error(f"ArXiv on-demand fetch failed: {e}")

        return result

    def preprocess_query(self, query: str) -> str:
        """
        Scientific-specific query preprocessing:
        - Strip conversational fluff for better retrieval
        - Add "research paper" context if not present
        """
        # Remove conversational prefixes
        strip_prefixes = [
            "can you find", "please summarize", "i want to know about",
            "tell me about", "search for", "find papers on",
        ]
        processed = query
        query_lower = query.lower()
        for prefix in strip_prefixes:
            if query_lower.startswith(prefix):
                processed = query[len(prefix):].strip()
                break

        return processed

    def postprocess_response(self, result: CRAGResult) -> CRAGResult:
        """
        Add citation block at the end of response if sources have ArXiv IDs.
        """
        arxiv_sources = [
            s for s in result.sources
            if "arxiv" in s.get("source", "").lower()
        ]

        if arxiv_sources:
            citations = []
            for i, src in enumerate(arxiv_sources, 1):
                parent_id = src.get("parent_id", "")
                citations.append(f"  [{i}] {parent_id}")

            if citations:
                result.response += "\n\n📚 **References:**\n" + "\n".join(citations)

        return result
''',

    'config/__init__.py': r'''
''',

    'config/prompts.py': r'''
"""
Prompt templates for all agents.
Stored as Python constants for Colab compatibility (no YAML dependency needed).
Each prompt uses {placeholders} for runtime injection.
"""

# ── Supervisor / Router ───────────────────────────────────────────────────────

SUPERVISOR_SYSTEM_PROMPT = """\
You are the Supervisor Agent of a Multi-Domain Intelligent Assistant.
Your job is to analyze user queries and route them to the correct specialist agent.

Available specialists:
1. **industrial** — Industrial equipment troubleshooting, PLC/SCADA systems, \
fault codes, maintenance procedures, automation diagnostics.
2. **recipe** — Cooking guidance, recipe search, ingredient substitutions, \
nutritional information, food preparation techniques.
3. **scientific** — Scientific paper summarization, ArXiv research queries, \
literature synthesis, academic concept explanation.

Routing rules:
- Classify the user's intent and select EXACTLY ONE specialist.
- If the query clearly belongs to one domain, route with high confidence.
- If the query is ambiguous or spans multiple domains, ask for clarification.
- If the query doesn't match any domain, route to "fallback" for web search.

Respond ONLY with valid JSON:
{{
  "domain": "<industrial|recipe|scientific|clarify|fallback>",
  "confidence": <0.0-1.0>,
  "reasoning": "<brief explanation of routing decision>"
}}
Reply with only that JSON object, on a single line if possible, with no explanation or prefix.

Few-shot examples:

User: "My Siemens S7-1200 PLC is showing fault code F0003"
{{"domain": "industrial", "confidence": 0.98, "reasoning": "PLC fault code troubleshooting"}}

User: "What can I substitute for buttermilk in a pancake recipe?"
{{"domain": "recipe", "confidence": 0.95, "reasoning": "Ingredient substitution query"}}

User: "Summarize recent transformer architecture papers on ArXiv"
{{"domain": "scientific", "confidence": 0.96, "reasoning": "Scientific literature request"}}

User: "What temperature should I cook chicken to avoid equipment failure?"
{{"domain": "clarify", "confidence": 0.40, "reasoning": "Ambiguous — could be cooking or industrial equipment"}}
"""

# ── Industrial Troubleshooting Agent ──────────────────────────────────────────

INDUSTRIAL_SYSTEM_PROMPT = """\
You are an Industrial Equipment Troubleshooting Specialist with expertise in:
- PLC programming and diagnostics (Siemens, Allen-Bradley, Mitsubishi)
- SCADA systems and HMI interfaces
- Motor drives, VFDs, and servo systems
- Industrial networking (PROFINET, EtherNet/IP, Modbus)
- Preventive and predictive maintenance procedures
- Fault code interpretation and resolution

Instructions:
1. Base your answers ONLY on the retrieved context documents provided below.
2. If the context is insufficient, say so explicitly — do not guess or hallucinate.
3. Always reference specific document sections, fault codes, or procedure numbers.
4. Use clear step-by-step troubleshooting format when applicable.
5. Flag any safety warnings prominently.

Retrieved Context:
{context}

User Query: {query}
"""

# ── Recipe Assistant Agent ────────────────────────────────────────────────────

RECIPE_SYSTEM_PROMPT = """\
You are a Recipe Assistant with expertise in cooking guidance, ingredient \
knowledge, and nutritional information.

Instructions:
1. Base your answers on the retrieved recipe context provided below.
2. When recommending recipes, mention key ingredients, prep time, and difficulty.
3. For substitution queries, explain WHY the substitute works (chemistry/texture).
4. Include relevant nutritional highlights when available.
5. If the context doesn't contain a suitable answer, say so clearly.

Retrieved Context:
{context}

User Query: {query}
"""

# ── Scientific Summarizer Agent ───────────────────────────────────────────────

SCIENTIFIC_SYSTEM_PROMPT = """\
You are a Scientific Paper Summarization Specialist. You synthesize academic \
research into clear, accurate summaries.

Instructions:
1. Base your summaries ONLY on the retrieved paper content below.
2. Structure summaries as: Objective → Method → Key Findings → Limitations.
3. When synthesizing multiple papers, identify agreements and contradictions.
4. Preserve technical accuracy — do not oversimplify domain-specific terms.
5. Always include citation information (authors, year, ArXiv ID).
6. If context is insufficient for a complete answer, state what's missing.

Retrieved Context:
{context}

User Query: {query}
"""

# ── Document Grader ───────────────────────────────────────────────────────────

GRADER_PROMPT = """\
You are a document relevance grader. Given a user query and a retrieved document, \
assess whether the document is relevant to answering the query.

Respond with ONLY valid JSON:
{{
  "relevance": "<relevant|irrelevant|ambiguous>",
  "score": <0.0-1.0>,
  "reasoning": "<brief explanation>"
}}

User Query: {query}

Retrieved Document:
{document}
"""

# Batch version: grade all documents in one call (much faster than N separate calls)
GRADER_BATCH_PROMPT = """\
You are a document relevance grader. Given a user query and several retrieved documents, \
assess each document's relevance to answering the query.

Respond with ONLY valid JSON. One object per document, in order (doc_0, doc_1, ...):
{{
  "grades": [
    {{ "relevance": "<relevant|irrelevant|ambiguous>", "score": <0.0-1.0>, "reasoning": "<brief>" }},
    ...
  ]
}}

User Query: {query}

Documents (numbered):
{documents_block}
"""

# ── Query Rewriter ────────────────────────────────────────────────────────────

REWRITER_PROMPT = """\
You are a query rewriting specialist. The original query failed to retrieve \
relevant documents. Rewrite it to improve retrieval while preserving the \
user's intent.

Strategies:
- Add domain-specific synonyms or technical terms
- Expand abbreviations
- Decompose compound questions into focused sub-queries
- Remove ambiguous phrasing

Original query: {query}
Domain: {domain}
Previous failed attempt context: {failure_context}

Respond with ONLY the rewritten query text, nothing else.
"""

# ── Hallucination Checker ─────────────────────────────────────────────────────

HALLUCINATION_CHECK_SYSTEM = """\
You are a hallucination detector.  Analyze a response against its source \
documents and decide whether every claim is grounded.

Respond with ONLY valid JSON — no other text:
{{"grounded": true/false, "confidence": 0.0-1.0, "issues": ["..."]}}
"""

HALLUCINATION_CHECK_USER = """\
Source Documents:
{sources}

Generated Response:
{response}

Return ONLY JSON."""
''',

    'config/settings.py': r'''
"""
Central configuration for the Multi-Domain Intelligent Assistant.
All model paths, hyperparameters, and constants live here.
Adjust for your Colab environment (GPU type, memory, etc.).
"""

from dataclasses import dataclass, field
from typing import Optional


# ── Model Configuration ──────────────────────────────────────────────────────

@dataclass
class LLMConfig:
    """Configuration for the primary language model."""
    preset: str = "gemma3_instruct_12b"             # KerasHub preset (~24GB bfloat16, fits A100 80GB)
    backend: str = "jax"                             # jax | torch | tensorflow
    dtype: str = "bfloat16"                          # bfloat16 for A100/L4
    max_new_tokens: int = 1024                       # full-generation budget (answers)
    short_max_new_tokens: int = 256                  # budget for routing, grading, hallucination checks
    temperature: float = 0.7
    top_k: int = 50
    top_p: float = 0.9
    fallback_preset: Optional[str] = None             # None = no fallback (avoids double OOM)


@dataclass
class EmbeddingConfig:
    """Configuration for the embedding model."""
    model_name: str = "thenlper/gte-large"
    dimension: int = 1024
    batch_size: int = 64
    max_seq_length: int = 512
    device: str = "cuda"
    trust_remote_code: bool = False


# ── Vector Store Configuration ────────────────────────────────────────────────

@dataclass
class VectorStoreConfig:
    """Configuration for ChromaDB vector store."""
    persist_directory: str = "./chroma_db"
    collections: dict = field(default_factory=lambda: {
        "industrial": "industrial_knowledge",
        "recipe": "recipe_knowledge",
        "scientific": "scientific_knowledge",
    })
    search_top_k: int = 5
    similarity_metric: str = "cosine"                # cosine | l2 | ip


# ── Chunking Configuration ────────────────────────────────────────────────────

@dataclass
class ChunkingConfig:
    """Configuration for document chunking."""
    chunk_size: int = 768                            # tokens (target midpoint)
    chunk_overlap_pct: float = 0.15                  # 15% overlap
    min_chunk_size: int = 512
    max_chunk_size: int = 1024
    separators: list = field(default_factory=lambda: [
        "\n\n", "\n", ". ", "! ", "? ", "; ", ", ", " "
    ])


# ── RAG / CRAG Configuration ─────────────────────────────────────────────────

@dataclass
class RAGConfig:
    """Configuration for the Corrective RAG pipeline."""
    max_rewrite_attempts: int = 2
    relevance_threshold: float = 0.7                 # doc grading cutoff
    confidence_threshold: float = 0.6                # hallucination check
    grading_labels: tuple = ("relevant", "irrelevant", "ambiguous")
    skip_hallucination_check: bool = False            # True = skip validation (saves one LLM call per query)


# ── Supervisor / Routing Configuration ────────────────────────────────────────

@dataclass
class DomainSpec:
    """Specification for a single domain (used for config-driven wiring)."""
    name: str
    agent_class: str          # dotted import path, e.g. "agents.industrial_agent.IndustrialAgent"
    prompt_key: str           # attribute name in config.prompts, e.g. "INDUSTRIAL_SYSTEM_PROMPT"
    collection: str           # vector store collection name


@dataclass
class SupervisorConfig:
    """Configuration for the supervisor/router agent."""
    routing_confidence_threshold: float = 0.75       # below → clarify
    domains: list = field(default_factory=lambda: [
        "industrial", "recipe", "scientific"
    ])
    domain_registry: list = field(default_factory=lambda: [
        DomainSpec("industrial", "agents.industrial_agent.IndustrialAgent",
                   "INDUSTRIAL_SYSTEM_PROMPT", "industrial_knowledge"),
        DomainSpec("recipe", "agents.recipe_agent.RecipeAgent",
                   "RECIPE_SYSTEM_PROMPT", "recipe_knowledge"),
        DomainSpec("scientific", "agents.scientific_agent.ScientificAgent",
                   "SCIENTIFIC_SYSTEM_PROMPT", "scientific_knowledge"),
    ])
    fallback_tool: str = "tavily_search"


# ── Aggregate Configuration ───────────────────────────────────────────────────

@dataclass
class ProjectConfig:
    """Top-level configuration combining all sub-configs."""
    llm: LLMConfig = field(default_factory=LLMConfig)
    embedding: EmbeddingConfig = field(default_factory=EmbeddingConfig)
    vector_store: VectorStoreConfig = field(default_factory=VectorStoreConfig)
    chunking: ChunkingConfig = field(default_factory=ChunkingConfig)
    rag: RAGConfig = field(default_factory=RAGConfig)
    supervisor: SupervisorConfig = field(default_factory=SupervisorConfig)


# Default config instance — import this everywhere
CONFIG = ProjectConfig()
''',

    'evaluation/__init__.py': r'''
''',

    'evaluation/metrics.py': r'''
"""
Evaluation Framework
====================
Test suite, metrics, and LLM-judge for evaluating the multi-agent system.
Covers: routing accuracy, retrieval precision, task completion, and
self-correction effectiveness.

Usage:
    from evaluation.metrics import Evaluator
    evaluator = Evaluator(workflow=workflow)
    results = evaluator.run_full_evaluation()
"""

from __future__ import annotations

import json
import logging
import time
from dataclasses import dataclass, field
from typing import Dict, List, Optional

from langchain_core.messages import HumanMessage

logger = logging.getLogger(__name__)


# ── Test Suite ────────────────────────────────────────────────────────────────

# 150 test queries: 50 per domain
# Each entry: (query, expected_domain, difficulty, description)

INDUSTRIAL_TEST_QUERIES = [
    ("My S7-1200 PLC is showing fault code F0003, what does it mean?", "industrial", "easy", "Direct fault code lookup"),
    ("The VFD on pump station 3 is tripping on overcurrent every morning", "industrial", "medium", "Intermittent fault diagnosis"),
    ("How do I configure PROFINET communication between two Siemens PLCs?", "industrial", "medium", "Configuration procedure"),
    ("What are the recommended PM intervals for Allen-Bradley servo drives?", "industrial", "easy", "Maintenance schedule"),
    ("Our SCADA system is losing connection to RTUs intermittently", "industrial", "hard", "Network troubleshooting"),
    ("Explain the difference between SIL 2 and SIL 3 safety requirements", "industrial", "medium", "Safety standards"),
    ("Motor bearing temperature is trending upward over the last month", "industrial", "medium", "Predictive maintenance"),
    ("How to reset a PowerFlex 525 drive after an F0063 fault?", "industrial", "easy", "Fault recovery procedure"),
    ("What causes ground fault alarms on a 480V distribution panel?", "industrial", "medium", "Electrical troubleshooting"),
    ("Best practices for PLC program backup and version control", "industrial", "easy", "Best practices"),
    # ... Add 40 more for full test suite
]

RECIPE_TEST_QUERIES = [
    ("What can I substitute for eggs in a chocolate cake recipe?", "recipe", "easy", "Common substitution"),
    ("I have chicken, rice, and bell peppers. What can I make?", "recipe", "medium", "Ingredient-based search"),
    ("How do I make a proper roux for gumbo?", "recipe", "easy", "Technique question"),
    ("What's the difference between baking soda and baking powder?", "recipe", "easy", "Ingredient knowledge"),
    ("Give me a gluten-free pasta recipe with under 500 calories", "recipe", "medium", "Dietary constraint search"),
    ("How long should I rest a steak after grilling?", "recipe", "easy", "Technique timing"),
    ("What's a good dairy-free alternative to heavy cream in soup?", "recipe", "medium", "Dietary substitution"),
    ("How do I properly temper chocolate for dipping?", "recipe", "hard", "Advanced technique"),
    ("What spices pair well with lamb?", "recipe", "easy", "Flavor pairing"),
    ("Can you give me a quick weeknight dinner recipe for 4 people?", "recipe", "medium", "Recommendation"),
    # ... Add 40 more for full test suite
]

SCIENTIFIC_TEST_QUERIES = [
    ("Summarize recent papers on transformer attention mechanisms", "scientific", "medium", "Topic synthesis"),
    ("What are the key findings from the latest RLHF research?", "scientific", "medium", "Current research"),
    ("Find papers about graph neural networks for drug discovery", "scientific", "easy", "Topic search"),
    ("What is the state of the art in protein structure prediction?", "scientific", "hard", "SOTA review"),
    ("Compare self-supervised vs supervised pretraining for NLP", "scientific", "hard", "Comparative analysis"),
    ("What datasets are commonly used for named entity recognition?", "scientific", "easy", "Resource query"),
    ("Explain the mixture of experts architecture in LLMs", "scientific", "medium", "Concept explanation"),
    ("Recent advances in federated learning for medical imaging", "scientific", "medium", "Domain-specific search"),
    ("What are the limitations of current RAG approaches?", "scientific", "medium", "Limitations analysis"),
    ("Find papers on energy-efficient training of large language models", "scientific", "easy", "Topic search"),
    # ... Add 40 more for full test suite
]

# Ambiguous/edge-case queries for routing stress testing
EDGE_CASE_QUERIES = [
    ("What temperature should I cook chicken to avoid equipment failure?", "clarify", "hard", "Ambiguous cross-domain"),
    ("How do I calibrate my oven thermometer?", "clarify", "medium", "Could be recipe or industrial"),
    ("What is the recipe for disaster recovery?", "clarify", "hard", "Metaphorical language"),
    ("Tell me about mixing processes in chemical plants", "industrial", "medium", "Industrial — not recipe mixing"),
    ("How does a neural network learn?", "scientific", "easy", "Clear scientific"),
]

ALL_TEST_QUERIES = (
    INDUSTRIAL_TEST_QUERIES
    + RECIPE_TEST_QUERIES
    + SCIENTIFIC_TEST_QUERIES
    + EDGE_CASE_QUERIES
)


# ── Metrics ───────────────────────────────────────────────────────────────────

@dataclass
class EvaluationResult:
    """Aggregated evaluation results."""
    routing_accuracy: float = 0.0
    routing_by_domain: Dict[str, float] = field(default_factory=dict)
    task_completion_rate: float = 0.0
    avg_latency_seconds: float = 0.0
    self_correction_rate: float = 0.0
    per_query_results: List[Dict] = field(default_factory=list)


class Evaluator:
    """
    Full evaluation suite for the multi-agent system.
    """

    def __init__(self, workflow, llm=None):
        self.workflow = workflow
        self.llm = llm  # For LLM-judge scoring

    def evaluate_routing(
        self,
        test_queries: Optional[List] = None,
    ) -> Dict:
        """
        Evaluate routing accuracy across all test queries.

        Returns accuracy overall and per-domain.
        """
        from orchestration.workflow_graph import run_query

        queries = test_queries or ALL_TEST_QUERIES
        correct = 0
        domain_correct = {}
        domain_total = {}
        results = []

        for query, expected_domain, difficulty, desc in queries:
            start = time.time()
            result = run_query(self.workflow, query)
            latency = time.time() - start

            actual_domain = result.get("domain", "")
            is_correct = actual_domain == expected_domain

            if is_correct:
                correct += 1

            domain_correct[expected_domain] = (
                domain_correct.get(expected_domain, 0) + (1 if is_correct else 0)
            )
            domain_total[expected_domain] = (
                domain_total.get(expected_domain, 0) + 1
            )

            results.append({
                "query": query,
                "expected": expected_domain,
                "actual": actual_domain,
                "correct": is_correct,
                "confidence": result.get("confidence", 0),
                "latency": latency,
                "difficulty": difficulty,
            })

        accuracy = correct / len(queries) if queries else 0.0
        per_domain = {
            d: domain_correct.get(d, 0) / domain_total.get(d, 1)
            for d in domain_total
        }

        return {
            "overall_accuracy": accuracy,
            "per_domain_accuracy": per_domain,
            "total_queries": len(queries),
            "correct": correct,
            "results": results,
        }

    def evaluate_llm_judge(
        self,
        query: str,
        response: str,
        domain: str,
    ) -> Dict:
        """
        Use LLM-as-judge to score a response for quality.

        Scores on: relevance, accuracy, completeness, clarity (each 1-5).
        """
        if not self.llm:
            return {"error": "No LLM provided for judging"}

        prompt = (
            f"You are evaluating an AI assistant's response quality.\n\n"
            f"Domain: {domain}\n"
            f"User Query: {query}\n"
            f"Assistant Response: {response}\n\n"
            f"Score the response on these dimensions (1-5 each):\n"
            f"1. Relevance — Does it address the query?\n"
            f"2. Accuracy — Is the information correct and grounded?\n"
            f"3. Completeness — Does it fully answer the question?\n"
            f"4. Clarity — Is it well-organized and easy to understand?\n\n"
            f"Respond with ONLY valid JSON:\n"
            f'{{"relevance": <1-5>, "accuracy": <1-5>, '
            f'"completeness": <1-5>, "clarity": <1-5>, '
            f'"overall": <1-5>, "feedback": "<brief explanation>"}}'
        )

        try:
            result = self.llm.invoke_and_parse_json(
                [HumanMessage(content=prompt)]
            )
            return result
        except Exception as e:
            return {"error": str(e)}

    # ── Verification: Functional ─────────────────────────────────────────

    def verify_functional(self, queries: Optional[List] = None) -> Dict:
        """Run representative queries across all domains, clarify, and fallback.

        Checks that every domain returns a non-empty response and a
        valid confidence value, and that the timing dict has a stable schema.
        """
        from orchestration.workflow_graph import run_query

        queries = queries or [
            ("My PowerFlex 525 drive shows fault F004", "industrial"),
            ("How do I make sourdough bread?", "recipe"),
            ("Summarize recent transformer papers", "scientific"),
            ("What temperature should I set the equipment/oven to?", "clarify"),
            ("Who won the 2024 Super Bowl?", "fallback"),
        ]

        results = []
        for query_text, expected in queries:
            result = run_query(self.workflow, query_text)
            timing = result.get("timing", {})
            crag_timing = timing.get("crag", {})

            checks = {
                "has_response": bool(result.get("response")),
                "domain_matches": result.get("domain") == expected,
                "confidence_valid": 0.0 <= result.get("confidence", -1) <= 1.0,
                "timing_schema_ok": all(
                    k in timing for k in ("total_s", "supervisor_s", "agent_s", "crag")
                ),
                "crag_schema_ok": all(
                    k in crag_timing
                    for k in ("retrieve_s", "grade_s", "rewrite_s", "generate_s", "validate_s", "total_s")
                ),
            }
            passed = all(checks.values())

            results.append({
                "query": query_text,
                "expected": expected,
                "actual": result.get("domain"),
                "passed": passed,
                "checks": checks,
            })
            status = "PASS" if passed else "FAIL"
            logger.info("Functional %s: %s → %s (expected %s)", status, query_text[:50], result.get("domain"), expected)

        return {
            "pass_rate": sum(r["passed"] for r in results) / len(results) if results else 0,
            "results": results,
        }

    # ── Verification: Reliability ──────────────────────────────────────

    @staticmethod
    def verify_reliability_json_parsing() -> Dict:
        """Test JSON extraction robustness with known-malformed LLM outputs.

        Does NOT require a running model -- tests the static parsing logic.
        """
        import json as _json
        from foundation.llm_wrapper import KerasHubChatModel

        test_cases = [
            ('{"domain":"industrial","confidence":0.9,"reasoning":"fault code"}',
             {"domain": "industrial"}),
            ('```json\n{"domain":"recipe","confidence":0.8,"reasoning":"cooking"}\n```',
             {"domain": "recipe"}),
            ('Sure! Here is the routing:\n{"domain":"scientific","confidence":0.95,"reasoning":"papers"}\nHope this helps!',
             {"domain": "scientific"}),
            ('user\nYou are the Supervisor Agent...',
             {"error": "parse_failed"}),
            ('[{"domain":"industrial","confidence":0.9,"reasoning":"only item"}]',
             {"domain": "industrial"}),
            ('{"confidence": "high", "domain": "recipe", "reasoning": "baking"}',
             {"domain": "recipe"}),
        ]

        results = []
        for raw_text, expected_subset in test_cases:
            # Directly test the parsing logic (text → dict) without model
            text = raw_text.strip()

            if "```" in text:
                parts = text.split("```")
                for part in parts[1::2]:
                    cleaned = part.strip()
                    if cleaned.startswith("json"):
                        cleaned = cleaned[4:].strip()
                    if cleaned.startswith("{") or cleaned.startswith("["):
                        text = cleaned
                        break

            text = text.strip()
            parsed = None
            try:
                p = _json.loads(text)
                if isinstance(p, list) and len(p) == 1 and isinstance(p[0], dict):
                    parsed = p[0]
                elif isinstance(p, dict):
                    parsed = p
            except _json.JSONDecodeError:
                pass

            if parsed is None:
                start = text.find("{")
                end = text.rfind("}")
                if start != -1 and end != -1 and end > start:
                    try:
                        parsed = _json.loads(text[start : end + 1])
                    except _json.JSONDecodeError:
                        pass

            if parsed is None:
                parsed = {"error": "parse_failed"}

            match = all(parsed.get(k) == v for k, v in expected_subset.items())
            results.append({
                "input": raw_text[:80],
                "expected_keys": expected_subset,
                "got": {k: parsed.get(k) for k in expected_subset},
                "match": match,
            })

        passed = sum(r["match"] for r in results)
        logger.info("Reliability JSON parsing: %d/%d passed", passed, len(results))
        return {"passed": passed, "total": len(results), "results": results}

    @staticmethod
    def verify_reliability_confidence_clamping() -> Dict:
        """Test confidence validation edge cases in supervisor."""
        from orchestration.supervisor import SupervisorAgent

        class FakeLLM:
            pass

        sup = SupervisorAgent.__new__(SupervisorAgent)
        sup.threshold = 0.75
        sup.valid_domains = {"industrial", "recipe", "scientific", "clarify", "fallback"}

        test_cases = [
            ({"confidence": 1.5, "domain": "industrial"}, 1.0),
            ({"confidence": -0.3, "domain": "recipe"}, 0.0),
            ({"confidence": "bad", "domain": "scientific"}, 0.0),
            ({"confidence": float("nan"), "domain": "industrial"}, 0.0),
            ({"confidence": 0.85, "domain": "industrial"}, 0.85),
            ({"confidence": 0.85, "domain": "unknown_domain"}, 0.85),
        ]

        results = []
        for input_dict, expected_conf in test_cases:
            routing = sup._build_routing_from_result(input_dict, "test query")
            conf_ok = abs(routing["confidence"] - expected_conf) < 1e-6
            domain_ok = routing["domain"] in sup.valid_domains
            results.append({
                "input": str(input_dict),
                "expected_confidence": expected_conf,
                "got_confidence": routing["confidence"],
                "confidence_ok": conf_ok,
                "domain_valid": domain_ok,
            })

        passed = sum(r["confidence_ok"] and r["domain_valid"] for r in results)
        logger.info("Reliability confidence clamping: %d/%d passed", passed, len(results))
        return {"passed": passed, "total": len(results), "results": results}

    # ── Verification: Performance ──────────────────────────────────────

    def verify_performance(self, queries: Optional[List] = None) -> Dict:
        """Measure p50/p95 latency and per-step breakdown.

        Run a set of queries and compute latency statistics.
        """
        from orchestration.workflow_graph import run_query

        queries = queries or [
            "My S7-1200 PLC is showing fault code F0003",
            "How do I make a proper roux for gumbo?",
            "Summarize recent papers on transformer attention",
        ]

        timings = []
        for q in queries:
            result = run_query(self.workflow, q)
            t = result.get("timing", {})
            timings.append({
                "query": q[:60],
                "total_s": t.get("total_s", 0),
                "supervisor_s": t.get("supervisor_s", 0),
                "agent_s": t.get("agent_s", 0),
                "crag_retrieve_s": t.get("crag", {}).get("retrieve_s", 0),
                "crag_grade_s": t.get("crag", {}).get("grade_s", 0),
                "crag_rewrite_s": t.get("crag", {}).get("rewrite_s", 0),
                "crag_generate_s": t.get("crag", {}).get("generate_s", 0),
                "crag_validate_s": t.get("crag", {}).get("validate_s", 0),
            })

        totals = sorted(t["total_s"] for t in timings)
        n = len(totals)

        stats = {
            "n_queries": n,
            "p50_s": totals[n // 2] if n else 0,
            "p95_s": totals[int(n * 0.95)] if n else 0,
            "min_s": totals[0] if n else 0,
            "max_s": totals[-1] if n else 0,
        }
        logger.info(
            "Performance: n=%d  p50=%.1fs  p95=%.1fs  min=%.1fs  max=%.1fs",
            stats["n_queries"], stats["p50_s"], stats["p95_s"],
            stats["min_s"], stats["max_s"],
        )
        return {"stats": stats, "per_query": timings}

    # ── Verification: Regression ───────────────────────────────────────

    def run_full_evaluation(self) -> EvaluationResult:
        """Run the complete evaluation suite (regression baseline)."""
        logger.info("Starting full evaluation...")

        routing_results = self.evaluate_routing()

        eval_result = EvaluationResult(
            routing_accuracy=routing_results["overall_accuracy"],
            routing_by_domain=routing_results["per_domain_accuracy"],
            per_query_results=routing_results["results"],
            avg_latency_seconds=(
                sum(r["latency"] for r in routing_results["results"])
                / len(routing_results["results"])
                if routing_results["results"] else 0
            ),
        )

        completed = sum(
            1 for r in routing_results["results"]
            if r.get("correct")
        )
        eval_result.task_completion_rate = (
            completed / len(routing_results["results"])
            if routing_results["results"] else 0
        )

        logger.info(
            f"Evaluation complete. Routing accuracy: {eval_result.routing_accuracy:.2%}"
        )
        return eval_result
''',

    'foundation/__init__.py': r'''
''',

    'foundation/embedding_service.py': r'''
"""
Embedding Service
=================
Wraps sentence-transformers (gte-large-en-v1.5) behind a clean interface.
Handles batching, device placement, and provides hooks for domain fine-tuning.

Usage:
    from foundation.embedding_service import EmbeddingService
    embedder = EmbeddingService()
    vectors = embedder.embed_documents(["text 1", "text 2"])
    query_vec = embedder.embed_query("search this")
"""

from __future__ import annotations

import logging
from typing import List, Optional

from langchain_core.embeddings import Embeddings

import sys
sys.path.append("..")
from config.settings import EmbeddingConfig, CONFIG

logger = logging.getLogger(__name__)


class EmbeddingService(Embeddings):
    """
    LangChain-compatible embedding service backed by sentence-transformers.
    Implements the Embeddings interface so it plugs directly into ChromaDB,
    retrievers, and any LangChain component expecting an embedder.
    """

    def __init__(self, config: Optional[EmbeddingConfig] = None):
        self.config = config or CONFIG.embedding
        self._model = None
        self._load_model()

    def _load_model(self):
        """Load the sentence-transformers model."""
        try:
            from sentence_transformers import SentenceTransformer

            logger.info(f"Loading embedding model: {self.config.model_name}")
            self._model = SentenceTransformer(
                self.config.model_name,
                device=self.config.device,
                trust_remote_code=self.config.trust_remote_code,
            )
            self._model.max_seq_length = self.config.max_seq_length
            logger.info(
                f"Embedding model loaded. Dimension: {self.config.dimension}, "
                f"Max seq length: {self.config.max_seq_length}"
            )
        except Exception as e:
            logger.error(f"Failed to load embedding model: {e}")
            raise

    # ── LangChain Embeddings Interface ────────────────────────────────────

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed a list of documents with batching."""
        all_embeddings = []
        for i in range(0, len(texts), self.config.batch_size):
            batch = texts[i : i + self.config.batch_size]
            embeddings = self._model.encode(
                batch,
                show_progress_bar=len(texts) > self.config.batch_size,
                normalize_embeddings=True,     # unit vectors for cosine sim
                convert_to_numpy=True,
            )
            all_embeddings.extend(embeddings.tolist())
        return all_embeddings

    def embed_query(self, text: str) -> List[float]:
        """Embed a single query string."""
        embedding = self._model.encode(
            text,
            normalize_embeddings=True,
            convert_to_numpy=True,
        )
        return embedding.tolist()

    # ── Domain Fine-Tuning Hook ───────────────────────────────────────────

    def fine_tune(
        self,
        train_pairs: List[tuple],
        epochs: int = 3,
        batch_size: int = 16,
        output_path: str = "./fine_tuned_embeddings",
    ):
        """
        Fine-tune the embedding model on domain-specific (query, positive) pairs
        using contrastive learning with hard negatives.

        Args:
            train_pairs: List of (query, positive_passage) tuples.
            epochs: Training epochs.
            batch_size: Training batch size.
            output_path: Where to save the fine-tuned model.
        """
        from sentence_transformers import InputExample, losses
        from torch.utils.data import DataLoader

        logger.info(
            f"Fine-tuning embeddings on {len(train_pairs)} pairs "
            f"for {epochs} epochs"
        )

        # Build training examples
        examples = [
            InputExample(texts=[q, p]) for q, p in train_pairs
        ]
        loader = DataLoader(examples, shuffle=True, batch_size=batch_size)

        # MultipleNegativesRankingLoss — standard for contrastive fine-tuning
        loss = losses.MultipleNegativesRankingLoss(self._model)

        self._model.fit(
            train_objectives=[(loader, loss)],
            epochs=epochs,
            output_path=output_path,
            show_progress_bar=True,
        )
        logger.info(f"Fine-tuned model saved to {output_path}")

    def load_fine_tuned(self, path: str):
        """Load a previously fine-tuned model."""
        from sentence_transformers import SentenceTransformer
        self._model = SentenceTransformer(path, device=self.config.device)
        logger.info(f"Loaded fine-tuned model from {path}")
''',

    'foundation/llm_wrapper.py': r'''
"""
KerasHub ↔ LangChain Bridge
============================
Custom wrapper that makes KerasHub models (Gemma 3, Gemma 2, Llama 3.1)
compatible with LangChain's BaseChatModel interface, enabling use inside
LangGraph agents.

This is one of the novel contributions of the project — no existing library
provides this integration.

Usage:
    from foundation.llm_wrapper import KerasHubChatModel
    llm = KerasHubChatModel(config=CONFIG.llm)
    response = llm.invoke([HumanMessage(content="Hello")])
"""

from __future__ import annotations

import json
import logging
import time
from typing import Any, Iterator, List, Optional

from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
)
from langchain_core.outputs import ChatGeneration, ChatResult

import sys
sys.path.append("..")
from config.settings import LLMConfig, CONFIG

logger = logging.getLogger(__name__)


# ── Prompt Formatters ─────────────────────────────────────────────────────────
# Each model family has its own control token format.

def _format_gemma(messages: List[BaseMessage]) -> str:
    """Format messages using Gemma 2 control tokens."""
    parts = []
    for msg in messages:
        if isinstance(msg, SystemMessage):
            # Gemma 2 doesn't have a system role — prepend to first user turn
            parts.append(f"<start_of_turn>user\n[System Instructions]\n{msg.content}\n")
        elif isinstance(msg, HumanMessage):
            # Avoid double user tag if system was prepended
            if parts and parts[-1].startswith("<start_of_turn>user\n[System"):
                parts[-1] += f"\n{msg.content}<end_of_turn>\n"
            else:
                parts.append(f"<start_of_turn>user\n{msg.content}<end_of_turn>\n")
        elif isinstance(msg, AIMessage):
            parts.append(f"<start_of_turn>model\n{msg.content}<end_of_turn>\n")
    # Open the model turn for generation
    parts.append("<start_of_turn>model\n")
    return "".join(parts)


def _format_llama(messages: List[BaseMessage]) -> str:
    """Format messages using Llama 3.1 control tokens."""
    parts = ["<|begin_of_text|>"]
    for msg in messages:
        if isinstance(msg, SystemMessage):
            parts.append(
                f"<|start_header_id|>system<|end_header_id|>\n\n"
                f"{msg.content}<|eot_id|>"
            )
        elif isinstance(msg, HumanMessage):
            parts.append(
                f"<|start_header_id|>user<|end_header_id|>\n\n"
                f"{msg.content}<|eot_id|>"
            )
        elif isinstance(msg, AIMessage):
            parts.append(
                f"<|start_header_id|>assistant<|end_header_id|>\n\n"
                f"{msg.content}<|eot_id|>"
            )
    parts.append("<|start_header_id|>assistant<|end_header_id|>\n\n")
    return "".join(parts)


# Map preset names to formatters
_FORMATTERS = {
    "gemma": _format_gemma,
    "llama": _format_llama,
}

def _get_formatter(preset: str):
    """Select the right prompt formatter based on model preset name."""
    preset_lower = preset.lower()
    for key, fn in _FORMATTERS.items():
        if key in preset_lower:
            return fn
    raise ValueError(
        f"No prompt formatter registered for preset '{preset}'. "
        f"Supported families: {list(_FORMATTERS.keys())}"
    )


# ── KerasHub Chat Model ──────────────────────────────────────────────────────

class KerasHubChatModel(BaseChatModel):
    """
    LangChain-compatible chat model backed by a KerasHub CausalLM.

    Handles:
    1. Prompt Construction — converts LangChain messages to model-specific
       control token format.
    2. Inference — triggers KerasHub generate() with configured sampling.
    3. Output Parsing — strips the prompt echo, extracts generated text.
    """

    # Pydantic fields (LangChain requires these to be class-level)
    config: LLMConfig = None
    model: Any = None           # keras_hub.models.CausalLM (set at init)
    _formatter: Any = None

    class Config:
        arbitrary_types_allowed = True

    def __init__(self, config: Optional[LLMConfig] = None, **kwargs):
        super().__init__(**kwargs)
        self.config = config or CONFIG.llm
        self._formatter = _get_formatter(self.config.preset)
        self._load_model()

    @staticmethod
    def _load_preset(keras_hub, preset: str, dtype: str):
        """Load a KerasHub model, using Gemma3CausalLM for gemma3 presets."""
        if "gemma3" in preset.lower():
            return keras_hub.models.Gemma3CausalLM.from_preset(preset, dtype=dtype)
        return keras_hub.models.CausalLM.from_preset(preset, dtype=dtype)

    def _load_model(self):
        """Load the KerasHub model with configured precision."""
        try:
            import os
            os.environ["KERAS_BACKEND"] = self.config.backend

            import keras_hub

            logger.info(f"Loading KerasHub model: {self.config.preset} "
                        f"(dtype={self.config.dtype})")
            self.model = self._load_preset(
                keras_hub, self.config.preset, self.config.dtype
            )
            logger.info("Model loaded successfully.")
        except Exception as e:
            logger.error(f"Failed to load primary model: {e}")
            if self.config.fallback_preset:
                logger.info(f"Attempting fallback: {self.config.fallback_preset}")
                self._formatter = _get_formatter(self.config.fallback_preset)
                import keras_hub
                self.model = self._load_preset(
                    keras_hub, self.config.fallback_preset, self.config.dtype
                )
            else:
                raise

    # ── LangChain Interface ───────────────────────────────────────────────

    @property
    def _llm_type(self) -> str:
        return "kerashub-chat"

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs,
    ) -> ChatResult:
        """Core generation method called by LangChain."""

        # 1. Format prompt
        prompt = self._formatter(messages)

        # Allow callers to override token budget (e.g. short tasks)
        token_budget = kwargs.get("max_new_tokens", self.config.max_new_tokens)

        # 2. Run inference (with optional timing for throughput diagnosis)
        t0 = time.perf_counter()
        raw_output = self.model.generate(
            prompt,
            max_length=len(prompt) + token_budget,
        )
        elapsed = time.perf_counter() - t0

        # 3. Strip prompt echo — KerasHub returns prompt + generation
        generated_text = raw_output
        if isinstance(raw_output, str) and raw_output.startswith(prompt):
            generated_text = raw_output[len(prompt):]
        elif isinstance(raw_output, str):
            # Fallback for Gemma 3: exact startswith may fail due to
            # whitespace/token differences.  Find the last model-turn
            # marker and take everything after it.
            marker = "<start_of_turn>model\n"
            last_model = raw_output.rfind(marker)
            if last_model != -1:
                generated_text = raw_output[last_model + len(marker):]

        # 4. Clean up trailing control tokens
        for token in ["<end_of_turn>", "<|eot_id|>", "<|end_of_text|>"]:
            generated_text = generated_text.split(token)[0]

        generated_text = generated_text.strip()

        # 5. Log throughput (approx: ~4 chars/token for English)
        est_tokens = max(1, len(generated_text) // 4)
        tps = est_tokens / elapsed if elapsed > 0 else 0
        logger.info(
            f"LLM call: {elapsed:.1f}s | ~{est_tokens} output tokens | "
            f"~{tps:.1f} tok/s"
        )

        return ChatResult(
            generations=[
                ChatGeneration(message=AIMessage(content=generated_text))
            ]
        )

    # ── Convenience ───────────────────────────────────────────────────────

    def invoke_and_parse_json(
        self, messages: List[BaseMessage], **kwargs
    ) -> dict:
        """Generate and parse a JSON response. Useful for grader/router.

        Uses the short token budget by default since structured JSON
        outputs (routing, grading, validation) are compact.
        """
        kwargs.setdefault("max_new_tokens", self.config.short_max_new_tokens)
        result = self.invoke(messages, **kwargs)
        text = result.content.strip()

        # Strip markdown code fences (handles ```json ... ``` and ``` ... ```)
        if "```" in text:
            parts = text.split("```")
            for part in parts[1::2]:  # odd-indexed parts are inside fences
                cleaned = part.strip()
                if cleaned.startswith("json"):
                    cleaned = cleaned[4:].strip()
                if cleaned.startswith("{") or cleaned.startswith("["):
                    text = cleaned
                    break

        text = text.strip()

        # Try direct parse
        try:
            parsed = json.loads(text)
            if isinstance(parsed, list) and len(parsed) == 1 and isinstance(parsed[0], dict):
                return parsed[0]
            if isinstance(parsed, dict):
                return parsed
        except json.JSONDecodeError:
            pass

        # Fallback: extract first { ... } substring
        start = text.find("{")
        end = text.rfind("}")
        if start != -1 and end != -1 and end > start:
            try:
                return json.loads(text[start : end + 1])
            except json.JSONDecodeError:
                pass

        logger.warning("JSON parse failed, raw output: %s", text[:200])
        return {"error": "parse_failed", "raw": text}
''',

    'foundation/vector_store.py': r'''
"""
Vector Store Service
====================
ChromaDB abstraction with multi-domain collection management.
Each domain (industrial, recipe, scientific) gets its own isolated collection.

Usage:
    from foundation.vector_store import VectorStoreService
    vs = VectorStoreService(embedding_service=embedder)
    vs.add_documents("industrial", docs)
    results = vs.search("industrial", "PLC fault code F0003", k=5)
"""

from __future__ import annotations

import logging
from functools import lru_cache
from typing import Dict, List, Optional, Tuple

import chromadb
from chromadb.config import Settings as ChromaSettings
from langchain_core.documents import Document

from foundation.embedding_service import EmbeddingService
from config.settings import VectorStoreConfig, CONFIG

logger = logging.getLogger(__name__)


class VectorStoreService:
    """
    Multi-domain vector store backed by ChromaDB.

    Key features:
    - One collection per domain for isolated retrieval
    - Consistent interface for add / search / delete
    - LangChain Document in/out for pipeline compatibility
    """

    def __init__(
        self,
        embedding_service: EmbeddingService,
        config: Optional[VectorStoreConfig] = None,
    ):
        self.config = config or CONFIG.vector_store
        self.embedder = embedding_service
        self._client = None
        self._collections: Dict[str, chromadb.Collection] = {}
        self._initialize()

    def _initialize(self):
        """Initialize ChromaDB client and create collections."""
        self._client = chromadb.Client(
            ChromaSettings(
                persist_directory=self.config.persist_directory,
                anonymized_telemetry=False,
            )
        )
        # Create or get each domain collection
        for domain, collection_name in self.config.collections.items():
            self._collections[domain] = self._client.get_or_create_collection(
                name=collection_name,
                metadata={"hnsw:space": self.config.similarity_metric},
            )
            logger.info(
                f"Collection '{collection_name}' ready "
                f"(count: {self._collections[domain].count()})"
            )

    # ── Core Operations ───────────────────────────────────────────────────

    def add_documents(
        self,
        domain: str,
        documents: List[Document],
        batch_size: int = 100,
    ) -> int:
        """
        Add LangChain Documents to a domain-specific collection.

        Returns the number of documents added.
        """
        self._validate_domain(domain)
        collection = self._collections[domain]

        texts = [doc.page_content for doc in documents]
        metadatas = [doc.metadata for doc in documents]
        ids = [
            doc.metadata.get("id", f"{domain}_{i}")
            for i, doc in enumerate(documents)
        ]

        # Embed and upsert in batches
        added = 0
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i : i + batch_size]
            batch_meta = metadatas[i : i + batch_size]
            batch_ids = ids[i : i + batch_size]
            batch_embeddings = self.embedder.embed_documents(batch_texts)

            collection.upsert(
                ids=batch_ids,
                embeddings=batch_embeddings,
                documents=batch_texts,
                metadatas=batch_meta,
            )
            added += len(batch_texts)

        logger.info(f"Added {added} documents to '{domain}' collection")
        return added

    def _embed_query_cached(self, query: str) -> Tuple[float, ...]:
        """Cache query embeddings to avoid re-encoding on CRAG retries."""
        return self.__embed_cache(query)

    @lru_cache(maxsize=128)
    def __embed_cache(self, query: str) -> Tuple[float, ...]:
        return tuple(self.embedder.embed_query(query))

    def search(
        self,
        domain: str,
        query: str,
        k: Optional[int] = None,
        where: Optional[dict] = None,
    ) -> List[Document]:
        """
        Semantic search within a domain collection.

        Returns LangChain Documents with metadata including similarity score.
        """
        self._validate_domain(domain)
        k = k or self.config.search_top_k
        collection = self._collections[domain]

        query_embedding = list(self._embed_query_cached(query))

        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=k,
            where=where,
            include=["documents", "metadatas", "distances"],
        )

        documents = []
        for doc_text, metadata, distance in zip(
            results["documents"][0],
            results["metadatas"][0],
            results["distances"][0],
        ):
            metadata["similarity_score"] = 1 - distance  # cosine distance → sim
            documents.append(
                Document(page_content=doc_text, metadata=metadata)
            )

        return documents

    def get_collection_stats(self, domain: str) -> dict:
        """Get stats for a domain collection."""
        self._validate_domain(domain)
        collection = self._collections[domain]
        return {
            "domain": domain,
            "collection_name": collection.name,
            "document_count": collection.count(),
        }

    def get_all_stats(self) -> List[dict]:
        """Get stats for all domain collections."""
        return [self.get_collection_stats(d) for d in self._collections]

    def clear_collection(self, domain: str):
        """Delete all documents in a domain collection."""
        self._validate_domain(domain)
        collection_name = self.config.collections[domain]
        self._client.delete_collection(collection_name)
        self._collections[domain] = self._client.get_or_create_collection(
            name=collection_name,
            metadata={"hnsw:space": self.config.similarity_metric},
        )
        logger.info(f"Cleared collection for domain '{domain}'")

    # ── Helpers ───────────────────────────────────────────────────────────

    def _validate_domain(self, domain: str):
        if domain not in self._collections:
            raise ValueError(
                f"Unknown domain '{domain}'. "
                f"Available: {list(self._collections.keys())}"
            )
''',

    'ingestion/__init__.py': r'''
''',

    'ingestion/chunking_pipeline.py': r'''
"""
Chunking Pipeline
=================
Splits documents into retrieval-optimized chunks with configurable overlap.
Industrial documents get extra preprocessing (abbreviation expansion,
equipment code normalization).

Usage:
    from ingestion.chunking_pipeline import ChunkingPipeline
    chunker = ChunkingPipeline()
    chunks = chunker.chunk_documents(docs, domain="industrial")
"""

from __future__ import annotations

import logging
import re
from typing import Dict, List, Optional

from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

from config.settings import ChunkingConfig, CONFIG

logger = logging.getLogger(__name__)


# ── Industrial Domain Preprocessors ───────────────────────────────────────────

# Common industrial abbreviations → expanded forms
INDUSTRIAL_ABBREVIATIONS: Dict[str, str] = {
    "PLC": "Programmable Logic Controller (PLC)",
    "HMI": "Human-Machine Interface (HMI)",
    "SCADA": "Supervisory Control and Data Acquisition (SCADA)",
    "VFD": "Variable Frequency Drive (VFD)",
    "RTU": "Remote Terminal Unit (RTU)",
    "DCS": "Distributed Control System (DCS)",
    "I/O": "Input/Output (I/O)",
    "OPC": "Open Platform Communications (OPC)",
    "SIL": "Safety Integrity Level (SIL)",
    "MTBF": "Mean Time Between Failures (MTBF)",
    "MTTR": "Mean Time To Repair (MTTR)",
    "PM": "Preventive Maintenance (PM)",
    "CM": "Corrective Maintenance (CM)",
    "NER": "Named Entity Recognition (NER)",
}

# Equipment code patterns (e.g., S7-1200, 6ES7-214, AB-1756)
EQUIPMENT_CODE_PATTERN = re.compile(
    r'\b([A-Z]{1,4}[-/]?\d{1,5}[-/]?\d{0,5}[A-Z]{0,3})\b'
)


def _preprocess_industrial(text: str) -> str:
    """
    Industrial-specific text preprocessing:
    1. Expand abbreviations (first occurrence only to avoid bloat)
    2. Normalize equipment codes for consistent retrieval
    3. Normalize whitespace
    """
    # Expand abbreviations — only first occurrence per abbreviation
    expanded = set()
    for abbrev, full in INDUSTRIAL_ABBREVIATIONS.items():
        if abbrev in text and abbrev not in expanded:
            # Replace first standalone occurrence
            text = re.sub(
                rf'\b{re.escape(abbrev)}\b',
                full,
                text,
                count=1,
            )
            expanded.add(abbrev)

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text


# Domain preprocessor registry
_PREPROCESSORS = {
    "industrial": _preprocess_industrial,
}


# ── Chunking Pipeline ────────────────────────────────────────────────────────

class ChunkingPipeline:
    """
    Document chunking with domain-aware preprocessing.

    Uses RecursiveCharacterTextSplitter under the hood with configurable
    chunk size, overlap, and separators. Industrial documents are preprocessed
    before chunking.
    """

    def __init__(self, config: Optional[ChunkingConfig] = None):
        self.config = config or CONFIG.chunking
        # Approximate tokens → chars (1 token ≈ 4 chars)
        self._char_chunk_size = self.config.chunk_size * 4
        self._char_overlap = int(self._char_chunk_size * self.config.chunk_overlap_pct)

        self._splitter = RecursiveCharacterTextSplitter(
            chunk_size=self._char_chunk_size,
            chunk_overlap=self._char_overlap,
            separators=self.config.separators,
            length_function=len,
            is_separator_regex=False,
        )

    def chunk_documents(
        self,
        documents: List[Document],
        domain: str = "general",
    ) -> List[Document]:
        """
        Chunk a list of documents, applying domain-specific preprocessing.

        Each output chunk inherits the parent document's metadata plus:
        - chunk_index: position within the parent
        - parent_id: ID of the source document
        """
        preprocessor = _PREPROCESSORS.get(domain)
        all_chunks = []

        for doc in documents:
            # Apply domain preprocessing if available
            text = doc.page_content
            if preprocessor:
                text = preprocessor(text)

            # Split into chunks
            text_chunks = self._splitter.split_text(text)

            parent_id = doc.metadata.get("id", doc.metadata.get("source", "unknown"))

            for i, chunk_text in enumerate(text_chunks):
                chunk_metadata = {
                    **doc.metadata,
                    "chunk_index": i,
                    "total_chunks": len(text_chunks),
                    "parent_id": parent_id,
                    "id": f"{parent_id}_chunk_{i}",
                    "domain": domain,
                }
                all_chunks.append(
                    Document(page_content=chunk_text, metadata=chunk_metadata)
                )

        logger.info(
            f"Chunked {len(documents)} documents into {len(all_chunks)} chunks "
            f"(domain={domain})"
        )
        return all_chunks

    def chunk_single(self, text: str, domain: str = "general") -> List[str]:
        """Chunk a single text string. Returns raw strings."""
        preprocessor = _PREPROCESSORS.get(domain)
        if preprocessor:
            text = preprocessor(text)
        return self._splitter.split_text(text)
''',

    'ingestion/document_loader.py': r'''
"""
Document Loader
===============
Unified loader that normalizes different source formats into LangChain Documents.
Supports: PDF, text/markdown, CSV (Food.com), ArXiv API, and raw strings.

Usage:
    from ingestion.document_loader import DocumentLoader
    loader = DocumentLoader()
    docs = loader.load_pdf("manual.pdf")
    docs = loader.load_food_csv("RAW_recipes.csv", max_rows=10000)
    docs = loader.load_arxiv("transformer attention mechanisms", max_results=5)
"""

from __future__ import annotations

import csv
import logging
import os
from pathlib import Path
from typing import List, Optional

from langchain_core.documents import Document

logger = logging.getLogger(__name__)


class DocumentLoader:
    """Unified document loader producing standardized LangChain Documents."""

    # ── PDF / Text Files ──────────────────────────────────────────────────

    @staticmethod
    def load_pdf(path: str, source_label: Optional[str] = None) -> List[Document]:
        """Load a PDF file using PyPDF2."""
        from PyPDF2 import PdfReader

        reader = PdfReader(path)
        docs = []
        label = source_label or Path(path).stem

        for i, page in enumerate(reader.pages):
            text = page.extract_text()
            if text and text.strip():
                docs.append(Document(
                    page_content=text.strip(),
                    metadata={
                        "source": label,
                        "file_path": path,
                        "page": i + 1,
                        "type": "pdf",
                    },
                ))
        logger.info(f"Loaded {len(docs)} pages from PDF: {path}")
        return docs

    @staticmethod
    def load_text(path: str, source_label: Optional[str] = None) -> List[Document]:
        """Load a plain text or markdown file."""
        with open(path, "r", encoding="utf-8") as f:
            content = f.read()
        label = source_label or Path(path).stem
        return [Document(
            page_content=content,
            metadata={
                "source": label,
                "file_path": path,
                "type": "text",
            },
        )]

    @staticmethod
    def load_directory(
        dir_path: str,
        extensions: tuple = (".pdf", ".txt", ".md"),
    ) -> List[Document]:
        """Load all supported files from a directory."""
        docs = []
        for root, _, files in os.walk(dir_path):
            for fname in sorted(files):
                fpath = os.path.join(root, fname)
                ext = Path(fname).suffix.lower()
                if ext == ".pdf":
                    docs.extend(DocumentLoader.load_pdf(fpath))
                elif ext in (".txt", ".md"):
                    docs.extend(DocumentLoader.load_text(fpath))
        logger.info(f"Loaded {len(docs)} documents from directory: {dir_path}")
        return docs

    # ── Food.com CSV ──────────────────────────────────────────────────────

    @staticmethod
    def load_food_csv(
        path: str,
        max_rows: Optional[int] = None,
    ) -> List[Document]:
        """
        Load Food.com RAW_recipes.csv into Documents.
        Each row becomes one Document with structured recipe content.
        """
        docs = []
        with open(path, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for i, row in enumerate(reader):
                if max_rows and i >= max_rows:
                    break

                # Build readable recipe text
                content = _format_recipe(row)
                if not content.strip():
                    continue

                docs.append(Document(
                    page_content=content,
                    metadata={
                        "source": "food.com",
                        "recipe_id": row.get("id", str(i)),
                        "name": row.get("name", ""),
                        "minutes": row.get("minutes", ""),
                        "n_ingredients": row.get("n_ingredients", ""),
                        "type": "recipe",
                        "id": f"recipe_{row.get('id', i)}",
                    },
                ))

        logger.info(f"Loaded {len(docs)} recipes from: {path}")
        return docs

    # ── ArXiv API ─────────────────────────────────────────────────────────

    @staticmethod
    def load_arxiv(
        query: str,
        max_results: int = 5,
        sort_by: str = "relevance",
    ) -> List[Document]:
        """
        Fetch papers from ArXiv API.
        Returns Documents with abstract + metadata (no full text by default).
        """
        import arxiv

        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=(
                arxiv.SortCriterion.Relevance
                if sort_by == "relevance"
                else arxiv.SortCriterion.SubmittedDate
            ),
        )

        docs = []
        for paper in search.results():
            content = (
                f"Title: {paper.title}\n\n"
                f"Authors: {', '.join(a.name for a in paper.authors)}\n\n"
                f"Abstract: {paper.summary}\n\n"
                f"Published: {paper.published.strftime('%Y-%m-%d')}\n"
                f"ArXiv ID: {paper.entry_id}"
            )
            docs.append(Document(
                page_content=content,
                metadata={
                    "source": "arxiv",
                    "arxiv_id": paper.entry_id,
                    "title": paper.title,
                    "authors": [a.name for a in paper.authors],
                    "published": paper.published.isoformat(),
                    "pdf_url": paper.pdf_url,
                    "type": "scientific",
                    "id": f"arxiv_{paper.entry_id.split('/')[-1]}",
                },
            ))

        logger.info(f"Fetched {len(docs)} papers from ArXiv for: '{query}'")
        return docs

    # ── Raw Text Helper ───────────────────────────────────────────────────

    @staticmethod
    def from_texts(
        texts: List[str],
        domain: str,
        source: str = "manual",
    ) -> List[Document]:
        """Create Documents from raw text strings."""
        return [
            Document(
                page_content=t,
                metadata={
                    "source": source,
                    "type": domain,
                    "id": f"{domain}_{i}",
                },
            )
            for i, t in enumerate(texts)
        ]


# ── Private Helpers ───────────────────────────────────────────────────────────

def _format_recipe(row: dict) -> str:
    """Format a Food.com CSV row into readable recipe text."""
    parts = []
    name = row.get("name", "Untitled Recipe")
    parts.append(f"Recipe: {name}")

    if row.get("minutes"):
        parts.append(f"Prep Time: {row['minutes']} minutes")

    if row.get("description"):
        parts.append(f"Description: {row['description']}")

    # Ingredients — stored as Python list string in CSV
    ingredients = row.get("ingredients", "")
    if ingredients:
        try:
            import ast
            ing_list = ast.literal_eval(ingredients)
            parts.append("Ingredients: " + ", ".join(ing_list))
        except (ValueError, SyntaxError):
            parts.append(f"Ingredients: {ingredients}")

    # Steps
    steps = row.get("steps", "")
    if steps:
        try:
            import ast
            step_list = ast.literal_eval(steps)
            parts.append("Steps:\n" + "\n".join(
                f"  {i+1}. {s}" for i, s in enumerate(step_list)
            ))
        except (ValueError, SyntaxError):
            parts.append(f"Steps: {steps}")

    if row.get("nutrition"):
        parts.append(f"Nutrition: {row['nutrition']}")

    return "\n".join(parts)
''',

    'ingestion/index_builder.py': r'''
"""
Index Builder
=============
Orchestrates the full ingestion pipeline: load → chunk → embed → store.
One-time setup per domain. Provides convenience methods for each domain type.

Usage:
    from ingestion.index_builder import IndexBuilder
    builder = IndexBuilder(vector_store=vs)

    # Industrial domain
    builder.index_industrial_docs("./data/manuals/")

    # Recipe domain
    builder.index_recipes("./data/RAW_recipes.csv", max_rows=50000)

    # Scientific domain (on-demand, not pre-indexed)
    builder.index_arxiv_papers("transformer attention", max_results=10)
"""

from __future__ import annotations

import logging
from typing import List, Optional

from langchain_core.documents import Document

from ingestion.document_loader import DocumentLoader
from ingestion.chunking_pipeline import ChunkingPipeline
from foundation.vector_store import VectorStoreService

logger = logging.getLogger(__name__)


class IndexBuilder:
    """
    End-to-end indexing orchestrator.
    Connects document loading, chunking, and vector storage.
    """

    def __init__(
        self,
        vector_store: VectorStoreService,
        chunker: Optional[ChunkingPipeline] = None,
    ):
        self.vector_store = vector_store
        self.chunker = chunker or ChunkingPipeline()
        self.loader = DocumentLoader()

    # ── Industrial Domain ─────────────────────────────────────────────────

    def index_industrial_docs(self, directory: str) -> int:
        """
        Index all PDFs and text files from an industrial manuals directory.
        Applies industrial-specific preprocessing during chunking.
        """
        logger.info(f"Indexing industrial documents from: {directory}")
        docs = self.loader.load_directory(directory)
        chunks = self.chunker.chunk_documents(docs, domain="industrial")
        count = self.vector_store.add_documents("industrial", chunks)
        logger.info(f"Industrial indexing complete: {count} chunks stored")
        return count

    def index_industrial_texts(self, texts: List[str], source: str = "manual") -> int:
        """Index raw text strings as industrial documents."""
        docs = self.loader.from_texts(texts, domain="industrial", source=source)
        chunks = self.chunker.chunk_documents(docs, domain="industrial")
        return self.vector_store.add_documents("industrial", chunks)

    # ── Recipe Domain ─────────────────────────────────────────────────────

    def index_recipes(self, csv_path: str, max_rows: Optional[int] = None) -> int:
        """
        Index Food.com recipes from CSV.
        Recipes are already semi-structured so chunking is lighter.
        """
        logger.info(f"Indexing recipes from: {csv_path}")
        docs = self.loader.load_food_csv(csv_path, max_rows=max_rows)
        # Recipes are typically short enough to be single chunks,
        # but we chunk anyway for consistency
        chunks = self.chunker.chunk_documents(docs, domain="recipe")
        count = self.vector_store.add_documents("recipe", chunks)
        logger.info(f"Recipe indexing complete: {count} chunks stored")
        return count

    # ── Scientific Domain ─────────────────────────────────────────────────

    def index_arxiv_papers(
        self,
        query: str,
        max_results: int = 10,
    ) -> int:
        """
        Fetch and index ArXiv papers for a given query.
        This is typically called on-demand rather than as a one-time setup.
        """
        logger.info(f"Fetching ArXiv papers for: '{query}'")
        docs = self.loader.load_arxiv(query, max_results=max_results)
        chunks = self.chunker.chunk_documents(docs, domain="scientific")
        count = self.vector_store.add_documents("scientific", chunks)
        logger.info(f"Scientific indexing complete: {count} chunks stored")
        return count

    # ── Generic ───────────────────────────────────────────────────────────

    def index_documents(
        self,
        documents: List[Document],
        domain: str,
    ) -> int:
        """Index pre-loaded documents into a specific domain."""
        chunks = self.chunker.chunk_documents(documents, domain=domain)
        return self.vector_store.add_documents(domain, chunks)

    def get_status(self) -> dict:
        """Return indexing status across all domains."""
        return {
            "collections": self.vector_store.get_all_stats(),
        }
''',

    'orchestration/__init__.py': r'''
''',

    'orchestration/state_schema.py': r'''
"""
State Schema
=============
Defines the shared state that flows through the LangGraph workflow.
All agents read from and write to this state object.
"""

from __future__ import annotations

from dataclasses import dataclass, field
from typing import Any, Dict, List, Literal, Optional, TypedDict

from langchain_core.messages import BaseMessage


class AgentState(TypedDict, total=False):
    """
    Shared state for the LangGraph supervisor workflow.

    This TypedDict defines every field that can flow through the graph.
    Nodes read what they need and write their outputs back.
    """

    # ── User Input ────────────────────────────────────────────────────────
    user_query: str                          # Original user query
    conversation_history: List[BaseMessage]  # Full conversation context

    # ── Routing ───────────────────────────────────────────────────────────
    routed_domain: str                       # "industrial" | "recipe" | "scientific" | "clarify" | "fallback"
    routing_confidence: float                # 0.0 - 1.0
    routing_reasoning: str                   # Why this domain was chosen

    # ── RAG State ─────────────────────────────────────────────────────────
    retrieved_documents: List[Dict]          # Raw retrieved docs
    relevant_documents: List[Dict]           # After grading
    current_query: str                       # May differ from user_query after rewriting
    rewrite_count: int                       # Number of query rewrites so far

    # ── Agent Output ──────────────────────────────────────────────────────
    agent_response: str                      # The generated response text
    agent_sources: List[Dict]                # Source attribution
    agent_confidence: float                  # Response confidence score
    agent_grounded: bool                     # Hallucination check result

    # ── Control Flow ──────────────────────────────────────────────────────
    escalated: bool                          # Whether agent escalated back
    escalation_reason: str                   # Why escalation happened
    needs_clarification: bool                # Supervisor requests more info
    clarification_message: str               # What to ask the user
    final_response: str                      # The response to return to user
    status: str                              # "routing" | "processing" | "complete" | "escalated"

    # ── Timing ────────────────────────────────────────────────────────
    timing_supervisor_s: float               # Supervisor routing time
    timing_agent_s: float                    # Agent (CRAG) processing time
    timing_clarify_s: float                  # Clarification generation time
    timing_crag_breakdown: Dict              # Per-step CRAG timing


def create_initial_state(query: str, history: Optional[List[BaseMessage]] = None) -> AgentState:
    """Create a fresh state for a new query."""
    return AgentState(
        user_query=query,
        conversation_history=history or [],
        routed_domain="",
        routing_confidence=0.0,
        routing_reasoning="",
        retrieved_documents=[],
        relevant_documents=[],
        current_query=query,
        rewrite_count=0,
        agent_response="",
        agent_sources=[],
        agent_confidence=0.0,
        agent_grounded=True,
        escalated=False,
        escalation_reason="",
        needs_clarification=False,
        clarification_message="",
        final_response="",
        status="routing",
    )
''',

    'orchestration/supervisor.py': r'''
"""
Supervisor Agent
================
Central router that classifies user intent and delegates to specialists.
Implements confidence-based routing with clarification fallback.

Usage:
    from orchestration.supervisor import SupervisorAgent
    supervisor = SupervisorAgent(llm=llm)
    routing = supervisor.route("My PLC is showing fault code F0003")
    # → {"domain": "industrial", "confidence": 0.98, ...}
"""

from __future__ import annotations

import logging
from typing import Dict

from langchain_core.messages import HumanMessage, SystemMessage

from config.prompts import SUPERVISOR_SYSTEM_PROMPT
from config.settings import CONFIG

logger = logging.getLogger(__name__)


class SupervisorAgent:
    """
    Supervisor/Router Agent.

    Analyzes user queries and routes to the appropriate specialist agent.
    Uses few-shot prompting and confidence scoring.
    """

    def __init__(self, llm):
        self.llm = llm
        self.threshold = CONFIG.supervisor.routing_confidence_threshold
        self.valid_domains = set(CONFIG.supervisor.domains + ["clarify", "fallback"])

    def route(self, query: str) -> Dict:
        """
        Classify and route a user query.

        Returns:
            {
                "domain": str,         # target domain or "clarify"/"fallback"
                "confidence": float,   # 0.0 - 1.0
                "reasoning": str,      # explanation
            }
        """
        system_msg = SystemMessage(content=SUPERVISOR_SYSTEM_PROMPT)
        user_prompt = (
            f'User query: "{query}"\n\n'
            "Output the routing as a single JSON object with keys "
            '"domain", "confidence", and "reasoning". '
            "Respond with ONLY that JSON object, with no extra text."
        )

        try:
            result = self.llm.invoke_and_parse_json(
                [system_msg, HumanMessage(content=user_prompt)]
            )

            if "error" in result:
                logger.warning(
                    "Supervisor parse failed (rich prompt), raw: %s",
                    result.get("raw", "")[:120],
                )
                # Second attempt: fall back to a simpler, JSON-only prompt.
                backup = self._route_with_simple_prompt(query)
                if backup is not None:
                    return backup
                return self._fallback_routing(query)

            return self._build_routing_from_result(result, query)

        except Exception as e:
            logger.error(f"Supervisor routing failed: {e}")
            backup = self._route_with_simple_prompt(query)
            if backup is not None:
                return backup
            return self._fallback_routing(query)

    def generate_clarification(self, query: str, routing: Dict) -> str:
        """Generate a clarification question when routing is ambiguous."""
        prompt = (
            f"The user asked: \"{query}\"\n\n"
            f"This query is ambiguous — it could relate to multiple domains. "
            f"Routing analysis: {routing.get('reasoning', 'unclear intent')}\n\n"
            f"Generate a SHORT, friendly clarification question to determine "
            f"the user's intent. Ask about the specific domain they need help with."
        )

        try:
            result = self.llm.invoke([HumanMessage(content=prompt)])
            return result.content.strip()
        except Exception:
            return (
                "I want to make sure I help you with the right thing. "
                "Could you clarify whether this is about industrial equipment, "
                "cooking/recipes, or scientific research?"
            )

    def _build_routing_from_result(self, result: Dict, query: str) -> Dict:
        """Common post-processing for LLM routing JSON."""
        domain = result.get("domain", "fallback")
        if domain not in self.valid_domains:
            logger.warning(f"Invalid domain '{domain}', using fallback")
            domain = "fallback"

        # Validate and clamp confidence to [0, 1]
        try:
            confidence = float(result.get("confidence", 0.0))
        except (TypeError, ValueError):
            confidence = 0.0
        if not (0.0 <= confidence <= 1.0) or confidence != confidence:  # NaN check
            logger.warning(f"Confidence out of range ({confidence}), clamping")
            confidence = max(0.0, min(1.0, confidence)) if confidence == confidence else 0.0

        if confidence < self.threshold and domain not in ("clarify", "fallback"):
            logger.info(
                f"Low confidence ({confidence:.2f} < {self.threshold}), "
                f"switching to clarification"
            )
            domain = "clarify"

        routing = {
            "domain": domain,
            "confidence": confidence,
            "reasoning": result.get("reasoning", ""),
        }

        logger.info(
            f"Routed to '{domain}' (confidence={confidence:.2f}): "
            f"{routing['reasoning'][:60]}"
        )
        return routing

    def _route_with_simple_prompt(self, query: str) -> Dict | None:
        """
        Backup routing path with a much simpler JSON-only prompt.

        Used when the rich supervisor prompt fails to produce valid JSON.
        """
        prompt = (
            "You are a router for a multi-domain assistant.\n"
            "Given a user query, choose exactly one domain from:\n"
            '  - "industrial"\n'
            '  - "recipe"\n'
            '  - "scientific"\n'
            '  - "clarify"\n'
            '  - "fallback"\n\n'
            "Respond ONLY with valid JSON in this format:\n"
            '{\n'
            '  \"domain\": \"<one of the above>\",\n'
            "  \"confidence\": <0.0-1.0>,\n"
            '  \"reasoning\": \"<brief explanation>\"\n'
            "}\n\n"
            f'User query: \"{query}\"'
        )

        try:
            result = self.llm.invoke_and_parse_json([HumanMessage(content=prompt)])
            if "error" in result:
                logger.warning(
                    "Simple supervisor prompt parse failed, raw: %s",
                    result.get("raw", "")[:120],
                )
                return None
            return self._build_routing_from_result(result, query)
        except Exception as e:
            logger.error(f"Simple supervisor routing failed: {e}")
            return None

    @staticmethod
    def _fallback_routing(query: str) -> Dict:
        """Fallback when LLM routing fails entirely."""
        return {
            "domain": "fallback",
            "confidence": 0.0,
            "reasoning": "Routing failed — using web search fallback",
        }
''',

    'orchestration/workflow_graph.py': r'''
"""
Workflow Graph
==============
LangGraph StateGraph that wires the supervisor, agents, and CRAG pipeline
into a complete executable workflow.

    START → supervisor_node → route_decision
        ├─ industrial → industrial_node → finalize
        ├─ recipe → recipe_node → finalize
        ├─ scientific → scientific_node → finalize
        ├─ clarify → clarify_node → END
        └─ fallback → fallback_node → finalize
    finalize → END

Usage:
    from orchestration.workflow_graph import build_workflow, run_query
    workflow = build_workflow(llm, vector_store, index_builder)
    result = run_query(workflow, "My PLC is showing fault F0003")
"""

from __future__ import annotations

import importlib
import logging
import time
from typing import Dict, Optional

from langgraph.graph import END, StateGraph

from orchestration.state_schema import AgentState, create_initial_state
from orchestration.supervisor import SupervisorAgent
from rag_core.crag_pipeline import CRAGPipeline
from foundation.vector_store import VectorStoreService
from ingestion.index_builder import IndexBuilder
from config.settings import CONFIG

logger = logging.getLogger(__name__)


def _import_class(dotted_path: str):
    """Import a class from a dotted path like 'agents.industrial_agent.IndustrialAgent'."""
    module_path, class_name = dotted_path.rsplit(".", 1)
    module = importlib.import_module(module_path)
    return getattr(module, class_name)


# ── Node Functions ────────────────────────────────────────────────────────────
# Each node is a function: State → State

def _make_supervisor_node(supervisor: SupervisorAgent):
    """Create the supervisor routing node."""
    def supervisor_node(state: AgentState) -> AgentState:
        query = state["user_query"]
        t0 = time.perf_counter()
        routing = supervisor.route(query)
        elapsed = time.perf_counter() - t0
        state["timing_supervisor_s"] = elapsed
        logger.info("⏱ Supervisor (route): %.1fs", elapsed)

        state["routed_domain"] = routing["domain"]
        state["routing_confidence"] = routing["confidence"]
        state["routing_reasoning"] = routing["reasoning"]
        state["status"] = "routing"

        return state
    return supervisor_node


def _make_agent_node(agent, domain_name: str):
    """Create a specialist agent execution node."""
    def agent_node(state: AgentState) -> AgentState:
        query = state["user_query"]
        state["status"] = "processing"

        t0 = time.perf_counter()
        result = agent.handle(query)
        elapsed = time.perf_counter() - t0
        state["timing_agent_s"] = elapsed
        state["timing_crag_breakdown"] = getattr(result, "timing_breakdown", {})
        logger.info("⏱ Agent (%s): %.1fs", domain_name, elapsed)

        state["agent_response"] = result.response
        state["agent_sources"] = result.sources
        state["agent_confidence"] = result.confidence
        state["agent_grounded"] = result.grounded
        state["escalated"] = result.escalated
        state["escalation_reason"] = result.escalation_reason

        return state
    return agent_node


def _make_clarify_node(supervisor: SupervisorAgent):
    """Create the clarification node."""
    def clarify_node(state: AgentState) -> AgentState:
        query = state["user_query"]
        routing = {
            "confidence": state.get("routing_confidence", 0),
            "reasoning": state.get("routing_reasoning", ""),
        }
        t0 = time.perf_counter()
        clarification = supervisor.generate_clarification(query, routing)
        state["timing_clarify_s"] = time.perf_counter() - t0
        logger.info("⏱ Clarify (LLM): %.1fs", state["timing_clarify_s"])

        state["needs_clarification"] = True
        state["clarification_message"] = clarification
        state["final_response"] = clarification
        state["status"] = "complete"

        return state
    return clarify_node


def _make_fallback_node(llm):
    """Create the web-search fallback node."""
    def fallback_node(state: AgentState) -> AgentState:
        # In a full implementation, this would use Tavily Search API
        state["final_response"] = (
            "This question falls outside my specialized domains "
            "(industrial troubleshooting, recipes, and scientific papers). "
            "I'd recommend searching the web for more information on this topic."
        )
        state["status"] = "complete"
        return state
    return fallback_node


def finalize_node(state: AgentState) -> AgentState:
    """Finalize the response — use agent output or escalation message."""
    if state.get("escalated") and not state.get("agent_response"):
        state["final_response"] = (
            f"I encountered difficulty answering your question. "
            f"Reason: {state.get('escalation_reason', 'Unknown')}. "
            f"Please try rephrasing or providing more details."
        )
    else:
        response = state.get("agent_response", "")
        # Add confidence warning if needed
        if state.get("escalated"):
            response += (
                "\n\n⚠️ *Note: I'm not fully confident in this answer. "
                "Please verify the information from the referenced sources.*"
            )
        state["final_response"] = response

    state["status"] = "complete"
    return state


# ── Routing Logic ─────────────────────────────────────────────────────────────

def _route_decision(state: AgentState) -> str:
    """Conditional edge: route to the appropriate agent node."""
    domain = state.get("routed_domain", "fallback")

    valid_routes = {spec.name for spec in CONFIG.supervisor.domain_registry}
    valid_routes |= {"clarify", "fallback"}
    if domain not in valid_routes:
        logger.warning(f"Unknown domain '{domain}', routing to fallback")
        return "fallback"

    return domain


# ── Graph Builder ─────────────────────────────────────────────────────────────

def build_workflow(
    llm,
    vector_store: VectorStoreService,
    index_builder: Optional[IndexBuilder] = None,
) -> StateGraph:
    """
    Build and compile the complete LangGraph workflow.

    Args:
        llm: LangChain-compatible chat model.
        vector_store: Initialized VectorStoreService.
        index_builder: Optional IndexBuilder for on-demand ArXiv fetching.

    Returns:
        Compiled LangGraph StateGraph ready for invocation.
    """
    # Initialize components
    supervisor = SupervisorAgent(llm)
    crag = CRAGPipeline(llm, vector_store)

    # Build graph
    graph = StateGraph(AgentState)
    graph.add_node("supervisor", _make_supervisor_node(supervisor))

    # Wire domain agents from config registry (config-driven extensibility)
    routing_map: Dict[str, str] = {}
    for spec in CONFIG.supervisor.domain_registry:
        agent_cls = _import_class(spec.agent_class)
        # ScientificAgent accepts an optional index_builder
        if spec.name == "scientific" and index_builder is not None:
            agent_instance = agent_cls(crag, index_builder)
        else:
            agent_instance = agent_cls(crag)
        graph.add_node(spec.name, _make_agent_node(agent_instance, spec.name))
        routing_map[spec.name] = spec.name

    graph.add_node("clarify", _make_clarify_node(supervisor))
    graph.add_node("fallback", _make_fallback_node(llm))
    graph.add_node("finalize", finalize_node)

    graph.set_entry_point("supervisor")

    routing_map["clarify"] = "clarify"
    routing_map["fallback"] = "fallback"
    graph.add_conditional_edges("supervisor", _route_decision, routing_map)

    # Domain agent nodes → finalize
    for spec in CONFIG.supervisor.domain_registry:
        graph.add_edge(spec.name, "finalize")
    graph.add_edge("fallback", "finalize")

    # Clarify and finalize → END
    graph.add_edge("clarify", END)
    graph.add_edge("finalize", END)

    # Compile
    compiled = graph.compile()
    logger.info("Workflow graph compiled successfully")
    return compiled


# ── Convenience Runner ────────────────────────────────────────────────────────

def run_query(workflow, query: str, history=None) -> Dict:
    """
    Run a query through the full workflow and return results.

    Returns:
        {
            "response": str,
            "domain": str,
            "confidence": float,
            "sources": list,
            "escalated": bool,
            "needs_clarification": bool,
            "timing": { "total_s", "supervisor_s", "agent_s", "crag": {...} },
        }
    """
    t0 = time.perf_counter()
    initial_state = create_initial_state(query, history)
    final_state = workflow.invoke(initial_state)
    total_s = time.perf_counter() - t0

    crag = final_state.get("timing_crag_breakdown") or {}

    timing = {
        "total_s": total_s,
        "supervisor_s": final_state.get("timing_supervisor_s", 0.0),
        "agent_s": final_state.get("timing_agent_s", 0.0),
        "clarify_s": final_state.get("timing_clarify_s", 0.0),
        "crag": {
            "retrieve_s": crag.get("retrieve_s", 0.0),
            "grade_s": crag.get("grade_s", 0.0),
            "rewrite_s": crag.get("rewrite_s", 0.0),
            "generate_s": crag.get("generate_s", 0.0),
            "validate_s": crag.get("validate_s", 0.0),
            "total_s": crag.get("total_s", 0.0),
        },
    }
    logger.info("⏱ Total query: %.1fs (%.1f min)", total_s, total_s / 60)

    return {
        "response": final_state.get("final_response", ""),
        "domain": final_state.get("routed_domain", ""),
        "confidence": final_state.get("routing_confidence", 0.0),
        "sources": final_state.get("agent_sources", []),
        "escalated": final_state.get("escalated", False),
        "needs_clarification": final_state.get("needs_clarification", False),
        "status": final_state.get("status", "unknown"),
        "timing": timing,
    }
''',

    'rag_core/__init__.py': r'''
''',

    'rag_core/crag_pipeline.py': r'''
"""
CRAG Pipeline (Corrective RAG)
===============================
Orchestrates the full self-correcting retrieval-augmented generation loop:

    Retrieve → Grade → Decide → Generate → Validate

If retrieved docs are irrelevant, rewrites the query and retries
(up to max_rewrite_attempts). If generation seems ungrounded,
flags for escalation to the supervisor.

This is the SHARED pipeline used by ALL specialized agents.

Usage:
    from rag_core.crag_pipeline import CRAGPipeline
    pipeline = CRAGPipeline(llm=llm, vector_store=vs)
    result = pipeline.run(query="PLC fault F0003", domain="industrial")
"""

from __future__ import annotations

import logging
import time
from dataclasses import dataclass, field
from typing import List, Optional

from langchain_core.documents import Document

from rag_core.retriever import Retriever
from rag_core.document_grader import DocumentGrader
from rag_core.query_rewriter import QueryRewriter
from rag_core.response_generator import ResponseGenerator
from rag_core.hallucination_checker import HallucinationChecker
from foundation.vector_store import VectorStoreService
from config.settings import CONFIG

logger = logging.getLogger(__name__)


@dataclass
class CRAGResult:
    """Result of a CRAG pipeline execution."""
    response: str = ""
    sources: list = field(default_factory=list)
    domain: str = ""
    grounded: bool = True
    confidence: float = 1.0
    attempts: int = 1
    escalated: bool = False
    escalation_reason: str = ""
    query_history: list = field(default_factory=list)
    grading_summary: dict = field(default_factory=dict)
    timing_breakdown: dict = field(default_factory=dict)
    per_attempt_timing: list = field(default_factory=list)  # [{retrieve_s, grade_s, rewrite_s}, ...]


class CRAGPipeline:
    """
    Self-correcting RAG pipeline implementing the CRAG pattern.

    The pipeline:
    1. RETRIEVE — semantic search in domain collection
    2. GRADE — LLM judges document relevance
    3. DECIDE —
         • Relevant docs found → proceed to generate
         • No relevant docs → rewrite query and retry (max N times)
         • Still nothing → escalate to supervisor
    4. GENERATE — produce grounded response with source attribution
    5. VALIDATE — hallucination check; escalate if ungrounded
    """

    def __init__(
        self,
        llm,
        vector_store: VectorStoreService,
    ):
        self.retriever = Retriever(vector_store)
        self.grader = DocumentGrader(llm)
        self.rewriter = QueryRewriter(llm)
        self.generator = ResponseGenerator(llm)
        self.checker = HallucinationChecker(llm)
        self.max_attempts = CONFIG.rag.max_rewrite_attempts + 1  # 1 initial + N rewrites

    def run(
        self,
        query: str,
        domain: str,
        k: Optional[int] = None,
    ) -> CRAGResult:
        """
        Execute the full CRAG loop.

        Args:
            query: User query (or already rewritten query).
            domain: Target domain for retrieval.
            k: Number of documents to retrieve per attempt.

        Returns:
            CRAGResult with response, sources, and pipeline metadata.
        """
        result = CRAGResult(domain=domain)
        current_query = query
        result.query_history.append(current_query)

        for attempt in range(1, self.max_attempts + 1):
            result.attempts = attempt
            attempt_timing: dict = {}
            logger.info(f"CRAG attempt {attempt}/{self.max_attempts} | "
                        f"Query: {current_query[:80]}")

            # ── Step 1: Retrieve ──────────────────────────────────────
            t0 = time.perf_counter()
            retrieved_docs = self.retriever.retrieve(domain, current_query, k=k)
            retrieve_elapsed = time.perf_counter() - t0
            attempt_timing["retrieve_s"] = retrieve_elapsed
            result.timing_breakdown["retrieve_s"] = result.timing_breakdown.get("retrieve_s", 0) + retrieve_elapsed

            if not retrieved_docs:
                logger.warning("No documents retrieved at all")
                if attempt < self.max_attempts:
                    t0 = time.perf_counter()
                    current_query = self._rewrite_and_retry(
                        current_query, domain, "No documents returned"
                    )
                    rewrite_elapsed = time.perf_counter() - t0
                    attempt_timing["rewrite_s"] = rewrite_elapsed
                    result.timing_breakdown["rewrite_s"] = result.timing_breakdown.get("rewrite_s", 0) + rewrite_elapsed
                    result.per_attempt_timing.append(attempt_timing)
                    result.query_history.append(current_query)
                    continue
                else:
                    result.per_attempt_timing.append(attempt_timing)
                    return self._escalate(
                        result, "No documents found after all attempts"
                    )

            # ── Step 2: Grade ─────────────────────────────────────────
            t0 = time.perf_counter()
            grading = self.grader.grade_documents(current_query, retrieved_docs)
            grade_elapsed = time.perf_counter() - t0
            attempt_timing["grade_s"] = grade_elapsed
            result.timing_breakdown["grade_s"] = result.timing_breakdown.get("grade_s", 0) + grade_elapsed
            result.grading_summary = {
                "relevant": len(grading["relevant"]),
                "irrelevant": len(grading["irrelevant"]),
                "ambiguous": len(grading["ambiguous"]),
            }

            # ── Step 3: Decide ────────────────────────────────────────
            if self.grader.has_sufficient_context(grading):
                context_docs = grading["relevant"] + grading["ambiguous"]
                logger.info(
                    f"Sufficient context found: {len(context_docs)} docs"
                )
                result.per_attempt_timing.append(attempt_timing)
                break
            else:
                logger.info("Insufficient relevant documents")
                if attempt < self.max_attempts:
                    failure_context = (
                        f"Retrieved {len(retrieved_docs)} docs but "
                        f"{len(grading['irrelevant'])} were irrelevant. "
                        f"Grades: {result.grading_summary}"
                    )
                    t0 = time.perf_counter()
                    current_query = self._rewrite_and_retry(
                        current_query, domain, failure_context
                    )
                    rewrite_elapsed = time.perf_counter() - t0
                    attempt_timing["rewrite_s"] = rewrite_elapsed
                    result.timing_breakdown["rewrite_s"] = result.timing_breakdown.get("rewrite_s", 0) + rewrite_elapsed
                    result.per_attempt_timing.append(attempt_timing)
                    result.query_history.append(current_query)
                    continue
                else:
                    result.per_attempt_timing.append(attempt_timing)
                    return self._escalate(
                        result,
                        "Could not find relevant documents after "
                        f"{self.max_attempts} attempts"
                    )
        else:
            return self._escalate(result, "Max attempts exhausted")

        # ── Step 4: Generate ──────────────────────────────────────────
        t0 = time.perf_counter()
        generation = self.generator.generate(query, context_docs, domain)
        result.timing_breakdown["generate_s"] = time.perf_counter() - t0
        result.response = generation["response"]
        result.sources = generation["sources"]

        # ── Step 5: Validate (optional; skip for faster inference) ─────
        if CONFIG.rag.skip_hallucination_check:
            result.grounded = True
            result.confidence = 1.0
            result.timing_breakdown["validate_s"] = 0.0
            logger.info("Hallucination check skipped (skip_hallucination_check=True)")
        else:
            t0 = time.perf_counter()
            validation = self.checker.check(result.response, context_docs)
            result.timing_breakdown["validate_s"] = time.perf_counter() - t0
            result.grounded = validation["grounded"]
            result.confidence = validation["confidence"]
            if validation["should_escalate"]:
                logger.warning(
                    f"Hallucination detected: {validation['issues']}"
                )
                result.escalated = True
                result.escalation_reason = (
                    f"Response may not be fully grounded. "
                    f"Issues: {validation['issues']}"
                )
            else:
                logger.info(
                    f"Response validated (confidence={result.confidence:.2f})"
                )

        result.timing_breakdown["total_s"] = (
            result.timing_breakdown.get("retrieve_s", 0)
            + result.timing_breakdown.get("grade_s", 0)
            + result.timing_breakdown.get("rewrite_s", 0)
            + result.timing_breakdown.get("generate_s", 0)
            + result.timing_breakdown.get("validate_s", 0)
        )
        logger.info(
            "⏱ CRAG timing: retrieve=%.1fs grade=%.1fs rewrite=%.1fs "
            "generate=%.1fs validate=%.1fs total=%.1fs (attempts=%d)",
            result.timing_breakdown.get("retrieve_s", 0),
            result.timing_breakdown.get("grade_s", 0),
            result.timing_breakdown.get("rewrite_s", 0),
            result.timing_breakdown.get("generate_s", 0),
            result.timing_breakdown.get("validate_s", 0),
            result.timing_breakdown["total_s"],
            result.attempts,
        )
        return result

    def _rewrite_and_retry(
        self, query: str, domain: str, failure_context: str
    ) -> str:
        """Rewrite query for the next retrieval attempt."""
        new_query = self.rewriter.rewrite(query, domain, failure_context)
        logger.info(f"Rewritten: '{query[:50]}' → '{new_query[:50]}'")
        return new_query

    @staticmethod
    def _escalate(result: CRAGResult, reason: str) -> CRAGResult:
        """Mark the result as needing supervisor escalation."""
        result.escalated = True
        result.escalation_reason = reason
        result.response = (
            "I wasn't able to find relevant information to answer your question "
            "from my knowledge base. Let me escalate this for further assistance."
        )
        logger.warning(f"Escalating to supervisor: {reason}")
        return result
''',

    'rag_core/document_grader.py': r'''
"""
Document Grader
===============
LLM-as-judge that evaluates whether retrieved documents are relevant
to the user's query. Core component of the CRAG self-correction loop.

Usage:
    from rag_core.document_grader import DocumentGrader
    grader = DocumentGrader(llm=llm)
    results = grader.grade_documents(query, retrieved_docs)
    relevant_docs = results["relevant"]
"""

from __future__ import annotations

import logging
from typing import Dict, List

from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, SystemMessage

from config.prompts import GRADER_PROMPT, GRADER_BATCH_PROMPT
from config.settings import CONFIG

logger = logging.getLogger(__name__)


class DocumentGrader:
    """
    Grades retrieved documents for relevance using LLM-as-judge.
    Categorizes each document as relevant, irrelevant, or ambiguous.
    """

    def __init__(self, llm):
        """
        Args:
            llm: A LangChain-compatible chat model (e.g., KerasHubChatModel).
        """
        self.llm = llm
        self.threshold = CONFIG.rag.relevance_threshold

    def grade_single(self, query: str, document: Document) -> dict:
        """
        Grade a single document's relevance to the query.

        Returns:
            {"relevance": "relevant"|"irrelevant"|"ambiguous",
             "score": float, "reasoning": str}
        """
        prompt = GRADER_PROMPT.format(
            query=query,
            document=document.page_content[:2000],  # truncate for context window
        )

        try:
            result = self.llm.invoke_and_parse_json(
                [HumanMessage(content=prompt)]
            )
            if "error" in result:
                # Parse failed — fall back to score-based grading
                logger.warning("Grader JSON parse failed, using fallback")
                return {
                    "relevance": "ambiguous",
                    "score": 0.5,
                    "reasoning": "Parse error — defaulting to ambiguous",
                }
            return result
        except Exception as e:
            logger.error(f"Grading failed: {e}")
            return {
                "relevance": "ambiguous",
                "score": 0.5,
                "reasoning": f"Error: {str(e)}",
            }

    def grade_documents_batch(
        self,
        query: str,
        documents: List[Document],
    ) -> Dict[str, List[Document]]:
        """
        Grade all documents in a single LLM call (faster than N separate calls).
        Falls back to per-document grading if batch parsing fails.
        """
        if not documents:
            return {"relevant": [], "irrelevant": [], "ambiguous": [], "grades": []}

        documents_block = "\n\n---\n\n".join(
            f"[Document {i}]\n{doc.page_content[:2000]}"
            for i, doc in enumerate(documents)
        )
        prompt = GRADER_BATCH_PROMPT.format(
            query=query,
            documents_block=documents_block,
        )

        try:
            result = self.llm.invoke_and_parse_json([HumanMessage(content=prompt)])
            if "error" in result or "grades" not in result:
                raise ValueError("Batch grader returned no grades")
            raw_grades = result["grades"]
            if not isinstance(raw_grades, list) or len(raw_grades) != len(documents):
                raise ValueError(
                    f"Expected {len(documents)} grades, got {len(raw_grades) if isinstance(raw_grades, list) else 'non-list'}"
                )
        except Exception as e:
            logger.warning(f"Batch grading failed ({e}), falling back to per-document grading")
            return self._grade_documents_sequential(query, documents)

        buckets = {"relevant": [], "irrelevant": [], "ambiguous": []}
        grades = []
        for i, doc in enumerate(documents):
            g = raw_grades[i] if i < len(raw_grades) else {}
            if not isinstance(g, dict):
                g = {"relevance": "ambiguous", "score": 0.5, "reasoning": "Invalid grade"}
            relevance = g.get("relevance", "ambiguous")
            if relevance not in CONFIG.rag.grading_labels:
                relevance = "ambiguous"
            buckets[relevance].append(doc)
            grades.append({"doc_index": i, "grade": g})
            logger.debug(f"Doc {i}: {relevance} (score={g.get('score', '?')})")

        logger.info(
            f"Grading results — relevant: {len(buckets['relevant'])}, "
            f"irrelevant: {len(buckets['irrelevant'])}, "
            f"ambiguous: {len(buckets['ambiguous'])}"
        )
        return {**buckets, "grades": grades}

    def _grade_documents_sequential(
        self,
        query: str,
        documents: List[Document],
    ) -> Dict[str, List[Document]]:
        """Per-document grading (slower, used as fallback)."""
        buckets = {"relevant": [], "irrelevant": [], "ambiguous": []}
        grades = []
        for i, doc in enumerate(documents):
            grade = self.grade_single(query, doc)
            relevance = grade.get("relevance", "ambiguous")
            if relevance not in CONFIG.rag.grading_labels:
                relevance = "ambiguous"
            buckets[relevance].append(doc)
            grades.append({"doc_index": i, "grade": grade})
        logger.info(
            f"Grading results — relevant: {len(buckets['relevant'])}, "
            f"irrelevant: {len(buckets['irrelevant'])}, ambiguous: {len(buckets['ambiguous'])}"
        )
        return {**buckets, "grades": grades}

    def grade_documents(
        self,
        query: str,
        documents: List[Document],
    ) -> Dict[str, List[Document]]:
        """
        Grade all retrieved documents and bucket them.
        Uses a single batch LLM call when possible for much faster inference.
        Returns:
            {
                "relevant": [Document, ...],
                "irrelevant": [Document, ...],
                "ambiguous": [Document, ...],
                "grades": [{"doc_index": int, "grade": dict}, ...],
            }
        """
        return self.grade_documents_batch(query, documents)

    def has_sufficient_context(self, grading_result: dict) -> bool:
        """Check if enough relevant documents were found to generate a response."""
        return len(grading_result["relevant"]) >= 1
''',

    'rag_core/hallucination_checker.py': r'''
"""
Hallucination Checker
=====================
Validates that a generated response is grounded in its source documents.
Flags unsupported claims and provides a confidence score.

Usage:
    from rag_core.hallucination_checker import HallucinationChecker
    checker = HallucinationChecker(llm=llm)
    result = checker.check(response_text, source_docs)
    if result["grounded"]:
        # Safe to return to user
"""

from __future__ import annotations

import logging
from typing import List

from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, SystemMessage

from config.prompts import HALLUCINATION_CHECK_SYSTEM, HALLUCINATION_CHECK_USER
from config.settings import CONFIG

logger = logging.getLogger(__name__)


class HallucinationChecker:
    """Validates response grounding against source documents."""

    def __init__(self, llm):
        self.llm = llm
        self.confidence_threshold = CONFIG.rag.confidence_threshold

    def check(
        self,
        response: str,
        source_documents: List[Document],
    ) -> dict:
        """
        Check if the response is grounded in the source documents.

        Returns:
            {
                "grounded": bool,
                "confidence": float,
                "issues": list[str],
                "should_escalate": bool,
            }
        """
        sources_text = "\n\n---\n\n".join(
            f"[Source {i+1}]\n{doc.page_content[:1500]}"
            for i, doc in enumerate(source_documents)
        )

        user_content = HALLUCINATION_CHECK_USER.format(
            sources=sources_text,
            response=response[:3000],
        )

        try:
            result = self.llm.invoke_and_parse_json([
                SystemMessage(content=HALLUCINATION_CHECK_SYSTEM),
                HumanMessage(content=user_content),
            ])
            if "error" in result:
                logger.warning("Hallucination check parse failed")
                return self._default_result()

            grounded = result.get("grounded", False)
            confidence = float(result.get("confidence", 0.5))
            issues = result.get("issues", [])

            return {
                "grounded": grounded,
                "confidence": confidence,
                "issues": issues,
                "should_escalate": not grounded or confidence < self.confidence_threshold,
            }

        except Exception as e:
            logger.error(f"Hallucination check failed: {e}")
            return self._default_result()

    @staticmethod
    def _default_result() -> dict:
        """Fail-safe: assume not fully grounded when check fails."""
        return {
            "grounded": False,
            "confidence": 0.5,
            "issues": ["Hallucination check could not be completed"],
            "should_escalate": True,
        }
''',

    'rag_core/query_rewriter.py': r'''
"""
Query Rewriter
==============
Rewrites queries that failed to retrieve relevant documents.
Uses the LLM to add domain-specific terms, expand abbreviations,
or decompose compound questions.

Usage:
    from rag_core.query_rewriter import QueryRewriter
    rewriter = QueryRewriter(llm=llm)
    new_query = rewriter.rewrite(query, domain="industrial", failure_context="...")
"""

from __future__ import annotations

import logging

from langchain_core.messages import HumanMessage

from config.prompts import REWRITER_PROMPT

logger = logging.getLogger(__name__)


class QueryRewriter:
    """Rewrites queries to improve retrieval on retry."""

    def __init__(self, llm):
        self.llm = llm

    def rewrite(
        self,
        query: str,
        domain: str = "general",
        failure_context: str = "No relevant documents found.",
    ) -> str:
        """
        Rewrite a query for better retrieval.

        Args:
            query: The original (or previously rewritten) query.
            domain: Target domain for terminology hints.
            failure_context: Why the previous retrieval failed.

        Returns:
            The rewritten query string.
        """
        prompt = REWRITER_PROMPT.format(
            query=query,
            domain=domain,
            failure_context=failure_context,
        )

        try:
            result = self.llm.invoke([HumanMessage(content=prompt)])
            rewritten = result.content.strip()
            # Sanity check — don't return empty or overly long rewrites
            if not rewritten or len(rewritten) > len(query) * 5:
                logger.warning("Rewrite was empty or too long, keeping original")
                return query
            logger.info(f"Query rewritten: '{query[:60]}' → '{rewritten[:60]}'")
            return rewritten
        except Exception as e:
            logger.error(f"Query rewrite failed: {e}")
            return query
''',

    'rag_core/response_generator.py': r'''
"""
Response Generator
==================
Produces grounded responses from query + relevant documents.
Injects domain-specific system prompts and formats source context.

Usage:
    from rag_core.response_generator import ResponseGenerator
    generator = ResponseGenerator(llm=llm)
    response = generator.generate(query, relevant_docs, domain="industrial")
"""

from __future__ import annotations

import logging
from typing import List

from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, SystemMessage

import config.prompts as prompt_module
from config.settings import CONFIG

logger = logging.getLogger(__name__)


def _build_domain_prompts() -> dict:
    """Build domain→prompt map from the config registry, falling back to legacy names."""
    prompts = {}
    for spec in CONFIG.supervisor.domain_registry:
        template = getattr(prompt_module, spec.prompt_key, None)
        if template:
            prompts[spec.name] = template
        else:
            logger.warning("No prompt template '%s' for domain '%s'", spec.prompt_key, spec.name)
    return prompts


_DOMAIN_PROMPTS = _build_domain_prompts()


class ResponseGenerator:
    """Generates grounded responses using retrieved context."""

    def __init__(self, llm):
        self.llm = llm

    def generate(
        self,
        query: str,
        documents: List[Document],
        domain: str,
    ) -> dict:
        """
        Generate a response grounded in the provided documents.

        Args:
            query: User's original query.
            documents: Relevant documents from retrieval.
            domain: Target domain for prompt selection.

        Returns:
            {
                "response": str,
                "sources": [{"source": str, "chunk_index": int}, ...],
            }
        """
        # Format context from documents
        context = self._format_context(documents)
        sources = self._extract_sources(documents)

        # Select domain prompt
        prompt_template = _DOMAIN_PROMPTS.get(domain)
        if not prompt_template:
            logger.warning(f"No prompt for domain '{domain}', using generic")
            prompt_template = (
                "Answer the user's query using ONLY the provided context.\n\n"
                "Retrieved Context:\n{context}\n\nUser Query: {query}"
            )

        prompt = prompt_template.format(context=context, query=query)

        try:
            result = self.llm.invoke([HumanMessage(content=prompt)])
            response_text = result.content.strip()
        except Exception as e:
            logger.error(f"Generation failed: {e}")
            response_text = (
                "I encountered an error generating a response. "
                "Please try rephrasing your question."
            )

        return {
            "response": response_text,
            "sources": sources,
            "domain": domain,
            "num_context_docs": len(documents),
        }

    @staticmethod
    def _format_context(documents: List[Document]) -> str:
        """Format documents into a numbered context string."""
        parts = []
        for i, doc in enumerate(documents, 1):
            source = doc.metadata.get("source", "unknown")
            parts.append(f"[Document {i} | Source: {source}]\n{doc.page_content}")
        return "\n\n---\n\n".join(parts)

    @staticmethod
    def _extract_sources(documents: List[Document]) -> list:
        """Extract source attribution from document metadata."""
        sources = []
        for doc in documents:
            sources.append({
                "source": doc.metadata.get("source", "unknown"),
                "chunk_index": doc.metadata.get("chunk_index", -1),
                "parent_id": doc.metadata.get("parent_id", ""),
                "similarity_score": doc.metadata.get("similarity_score", 0.0),
            })
        return sources
''',

    'rag_core/retriever.py': r'''
"""
Retriever
=========
Thin wrapper around VectorStoreService.search() that provides a consistent
interface for the CRAG pipeline. Adds logging and query preprocessing.

Usage:
    from rag_core.retriever import Retriever
    retriever = Retriever(vector_store=vs)
    docs = retriever.retrieve("industrial", "PLC fault F0003", k=5)
"""

from __future__ import annotations

import logging
from typing import List, Optional

from langchain_core.documents import Document

from foundation.vector_store import VectorStoreService
from config.settings import CONFIG

logger = logging.getLogger(__name__)


class Retriever:
    """
    Domain-aware document retriever.
    Routes queries to the correct vector store collection.
    """

    def __init__(self, vector_store: VectorStoreService):
        self.vector_store = vector_store

    def retrieve(
        self,
        domain: str,
        query: str,
        k: Optional[int] = None,
        filters: Optional[dict] = None,
    ) -> List[Document]:
        """
        Retrieve top-k documents for a query from the specified domain.

        Args:
            domain: Target domain ("industrial", "recipe", "scientific").
            query: The user query or rewritten query.
            k: Number of results (defaults to config).
            filters: Optional ChromaDB where-clause for metadata filtering.

        Returns:
            List of Documents with similarity_score in metadata.
        """
        k = k or CONFIG.vector_store.search_top_k

        logger.info(f"Retrieving top-{k} from '{domain}' for: {query[:80]}...")
        docs = self.vector_store.search(domain, query, k=k, where=filters)
        logger.info(
            f"Retrieved {len(docs)} documents "
            f"(scores: {[f'{d.metadata.get('similarity_score', 0):.3f}' for d in docs]})"
        )
        return docs

    def retrieve_with_scores(
        self,
        domain: str,
        query: str,
        k: Optional[int] = None,
    ) -> List[tuple]:
        """Retrieve documents as (Document, score) tuples."""
        docs = self.retrieve(domain, query, k=k)
        return [
            (doc, doc.metadata.get("similarity_score", 0.0))
            for doc in docs
        ]
''',

}

for rel_path, content in _FILES.items():
    dst = os.path.join(PROJECT_ROOT, rel_path)
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    with open(dst, 'w') as f:
        f.write(content)
    print(f'  \u2713 {rel_path}')

print(f'\nWrote {len(_FILES)} files to {PROJECT_ROOT}')

for mod_name in list(sys.modules.keys()):
    if any(pkg in mod_name for pkg in [
        'config', 'foundation', 'rag_core',
        'orchestration', 'agents', 'ingestion', 'evaluation',
    ]):
        del sys.modules[mod_name]

from config.settings import CONFIG
print(f'\nConfig reloaded. LLM preset: {CONFIG.llm.preset}')
print(f'Fallback preset: {CONFIG.llm.fallback_preset}')
print(f'Embedding model: {CONFIG.embedding.model_name}')


In [14]:
import os, sys
from config.settings import CONFIG, LLMConfig
from foundation.llm_wrapper import KerasHubChatModel
from foundation.embedding_service import EmbeddingService
from foundation.vector_store import VectorStoreService

# Print configuration
print(f"LLM: {CONFIG.llm.preset}")
print(f"Embeddings: {CONFIG.embedding.model_name}")
print(f"Domains: {CONFIG.supervisor.domains}")

from google.colab import userdata
import os

os.environ["KAGGLE_USERNAME"] = "roshcs"
os.environ["KAGGLE_KEY"] = "781502c08d14d778d3d23d4e58ca0751"

# os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
# os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

LLM: gemma3_instruct_12b
Embeddings: Alibaba-NLP/gte-large-en-v1.5
Domains: ['industrial', 'recipe', 'scientific']


In [15]:
# Now try loading
from foundation.llm_wrapper import KerasHubChatModel
from config.settings import CONFIG

llm = KerasHubChatModel(config=CONFIG.llm)
print("✅ LLM loaded")


✅ LLM loaded


In [16]:
from foundation.embedding_service import EmbeddingService

embedder = EmbeddingService()

test_vec = embedder.embed_query("PLC fault code")
print(f"✅ Embedding dimension: {len(test_vec)}")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

configuration.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
ERROR:foundation.embedding_service:Failed to load embedding model: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 'NewConfig {
  "architectures": [
    "NewModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "auto_map": {
    "AutoConfig": "Alibaba-NLP/new-impl--configuration.NewConfig",
    "AutoModel": "Alibaba-NLP/new-impl--modeling.NewModel",
    "AutoModelForMaskedLM": "Alibaba-NLP/new-impl--modeling.NewForMaskedLM",
    "AutoModelForMultipleChoice": "Alibaba-NLP/new-impl--modeling.NewForMultipleChoice",
    "AutoModelForQuestionAnswering": "Alibaba-NLP/new-impl--modeling.NewForQuestionAnswering",
    "AutoModelForSequenceClassification": "Alibaba-NLP/new-impl--modeling.NewForSequenceClassification",

OSError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 'NewConfig {
  "architectures": [
    "NewModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "auto_map": {
    "AutoConfig": "Alibaba-NLP/new-impl--configuration.NewConfig",
    "AutoModel": "Alibaba-NLP/new-impl--modeling.NewModel",
    "AutoModelForMaskedLM": "Alibaba-NLP/new-impl--modeling.NewForMaskedLM",
    "AutoModelForMultipleChoice": "Alibaba-NLP/new-impl--modeling.NewForMultipleChoice",
    "AutoModelForQuestionAnswering": "Alibaba-NLP/new-impl--modeling.NewForQuestionAnswering",
    "AutoModelForSequenceClassification": "Alibaba-NLP/new-impl--modeling.NewForSequenceClassification",
    "AutoModelForTokenClassification": "Alibaba-NLP/new-impl--modeling.NewForTokenClassification"
  },
  "classifier_dropout": null,
  "dtype": "float32",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "lay

In [ ]:
from foundation.vector_store import VectorStoreService

vs = VectorStoreService(embedding_service=embedder)
print("✅ Vector store initialized")
print(vs.get_all_stats())

In [ ]:
from ingestion.index_builder import IndexBuilder
from ingestion.document_loader import DocumentLoader
from ingestion.chunking_pipeline import ChunkingPipeline

builder = IndexBuilder(vector_store=vs)

# ── Industrial: Rockwell Automation / Allen-Bradley ──
sample_industrial = [
    """Fault Code F002 - Allen-Bradley PowerFlex 525 VFD: Auxiliary Input.
    Description: An external fault has been detected via the auxiliary input terminal.
    Possible Causes:
    1. External device connected to auxiliary input is reporting a fault condition
    2. Wiring issue on the auxiliary digital input terminal
    3. Incorrect parameter configuration for the auxiliary input function
    Troubleshooting Steps:
    1. Check the external device connected to the auxiliary input for faults
    2. Verify wiring connections on terminal block TB2
    3. Check parameter A070 [Fault Config 2] for correct auxiliary input configuration
    4. Inspect for damaged or loose wiring on the digital input terminals
    Resolution: Clear the external fault condition and reset the drive by cycling the Enable input or pressing the Stop/Reset key on the HIM.""",

    """Fault Code F004 - Allen-Bradley PowerFlex 525 VFD: Undervoltage.
    Description: The DC bus voltage has dropped below the undervoltage trip level.
    Possible Causes:
    1. Input power supply voltage too low or momentary power dip
    2. Incoming power supply fuses blown or circuit breaker tripped
    3. Loose connections on input power terminals R/L1, S/L2, T/L3
    4. DC bus capacitors degraded (check capacitor health indicator LED)
    Troubleshooting Steps:
    1. Measure incoming line voltage at drive terminals - must be within nameplate rating +/-10%
    2. Check all three phases for voltage balance (max 3% imbalance)
    3. Inspect and tighten input power terminal connections (torque to 1.4 Nm)
    4. Check parameter A531 [DC Bus Voltage] for current reading
    5. Review fault log in parameter A700-A706 for fault history and timestamps
    Resolution: Restore proper input voltage. If capacitors are degraded, replace the drive.
    SAFETY: Disconnect and lockout/tagout all power sources before inspecting terminals.
    Wait 5 minutes for DC bus capacitors to discharge before servicing.""",

    """Fault Code F005 - Allen-Bradley PowerFlex 525 VFD: Overvoltage.
    Description: The DC bus voltage has exceeded the overvoltage trip level.
    Possible Causes:
    1. Input line voltage exceeds drive nameplate rating
    2. Excessive regenerative energy from motor deceleration (overhauling load)
    3. Deceleration time too short for the load inertia
    4. Dynamic brake resistor circuit failure or incorrect sizing
    Troubleshooting Steps:
    1. Measure input voltage - must not exceed 528 VAC for 480V class drives
    2. Increase deceleration time in parameter A092 [Decel Time 1]
    3. Enable bus regulator: set parameter A540 [Bus Reg Mode] to option 1 (Enabled)
    4. If using dynamic braking, check DB resistor connections and resistance value
    5. Check parameter A531 [DC Bus Voltage] - nominal is ~650V for 480V input
    Resolution: Reduce input voltage or extend deceleration time. Add DB resistor for overhauling loads.""",

    """Allen-Bradley CompactLogix 5380 Controller - EtherNet/IP Configuration Guide.
    Model: 1769-L33ER CompactLogix 5380
    Prerequisites: RSLogix 5000 v32+ or Studio 5000 Logix Designer v32+
    Step 1: Create new project - select 1769-L33ER controller, revision 32+
    Step 2: Configure controller IP address via USB connection using BOOTP/DHCP tool
    Step 3: Add EtherNet/IP module in I/O Configuration tree
    Step 4: Right-click controller > Properties > set IP address (e.g., 192.168.1.10)
    Step 5: Add remote I/O devices - right-click EtherNet/IP > New Module
    Step 6: Configure Produced/Consumed tags for controller-to-controller communication
    Step 7: Download program and verify connection indicators:
      - OK LED: Solid green = running, Flashing green = program mode
      - ENET LED: Solid green = has connections, Flashing green = no connections
    Common Issue: If ENET LED is off, check IP config and network switch connection.
    Use RSLinx Classic > RSWho to verify controller appears on EtherNet/IP network.""",

    """Allen-Bradley PanelView Plus 7 HMI - Troubleshooting Communication Failures.
    Symptom: PanelView displays "Controller not found" or shows stale data.
    Possible Causes:
    1. Incorrect IP address or subnet mask configuration
    2. EtherNet/IP cable disconnected or faulty
    3. RSLinx Enterprise communication path misconfigured
    4. Controller is in Program mode instead of Run mode
    Troubleshooting Steps:
    1. On PanelView: press Terminal Settings > Networks > verify IP address
    2. Ping the controller IP from PanelView diagnostics screen
    3. In FactoryTalk View Studio: check Communication Setup > verify shortcut path
    4. Ensure controller and HMI are on same subnet (e.g., both 192.168.1.x/24)
    5. Check Ethernet cable and switch port LEDs for link activity
    6. Review controller mode - must be in Run or Remote Run for live data
    Resolution: Correct IP addressing and verify physical network connectivity.
    If using managed switch, verify VLANs are not isolating devices.""",

    """Preventive Maintenance Schedule - Allen-Bradley PowerFlex 755 Drive System.
    Weekly: Check drive status LEDs and HIM display for active alarms
    Monthly: Inspect cooling fans for proper operation, clean air filters
    Quarterly: Verify DC bus voltage (parameter 15 - Bus Voltage), check capacitor
    formation indicator, tighten all power and control terminal connections
    Semi-Annually: Thermal scan of power connections, verify ground fault monitoring,
    backup drive parameters using Connected Components Workbench (CCW)
    Annually: Full parameter backup, firmware version audit against Rockwell
    compatibility matrix, capacitor health test, clean power module heat sinks
    Every 5 Years: Replace cooling fans (recommended life), capacitor reformation
    MTBF: PowerFlex 755 rated at approximately 200,000 hours (28+ years) at 40C ambient.
    Critical Spares to Stock: Cooling fan assembly (KIT-F755FAN), control board fuse,
    fiber optic cables for multi-axis configurations.""",
]

count = builder.index_industrial_texts(sample_industrial, source="rockwell_automation")
print(f"✅ Indexed {count} industrial chunks")

# ── Recipe sample data ──
sample_recipes = [
    """Recipe: Classic Chocolate Chip Cookies
    Prep Time: 45 minutes. Ingredients: 2 1/4 cups flour, 1 tsp baking soda,
    1 tsp salt, 1 cup butter softened, 3/4 cup sugar, 3/4 cup brown sugar,
    2 large eggs, 2 tsp vanilla, 2 cups chocolate chips.
    Egg Substitutes: Use 1/4 cup unsweetened applesauce per egg,
    or 1 mashed banana per egg, or 3 tbsp aquafaba per egg.""",

    """Recipe: Quick Chicken Stir-Fry
    Prep Time: 20 minutes. Ingredients: 1 lb chicken breast, 2 bell peppers,
    1 cup rice, 3 tbsp soy sauce, 1 tbsp sesame oil, 2 cloves garlic, ginger.
    Steps: Cook rice. Slice chicken. Heat oil in wok over high heat.
    Stir-fry chicken 5-6 min. Add vegetables and garlic. Add soy sauce.
    Nutrition: ~450 calories per serving, 35g protein.""",

    """Recipe: Homemade Margherita Pizza
    Prep Time: 90 minutes (including dough rise). Ingredients: 3 cups flour,
    1 packet yeast, 1 cup warm water, 2 tbsp olive oil, 1 tsp salt, 1 tsp sugar,
    1 cup crushed San Marzano tomatoes, 8 oz fresh mozzarella, fresh basil.
    Steps: Mix dough, let rise 1 hour. Preheat oven to 475F. Stretch dough,
    add sauce and cheese. Bake 12-15 min until crust is golden.
    Gluten-Free Alternative: Substitute 3 cups gluten-free flour blend plus
    1 tsp xanthan gum for regular flour.""",
]

recipe_docs = DocumentLoader.from_texts(sample_recipes, "recipe", "demo_recipes")
chunker = ChunkingPipeline()
recipe_chunks = chunker.chunk_documents(recipe_docs, domain="recipe")
count = vs.add_documents("recipe", recipe_chunks)
print(f"✅ Indexed {count} recipe chunks")

# ── Scientific sample data ──
sample_scientific = [
    """Title: Attention Is All You Need
    Authors: Vaswani, Shazeer, Parmar, Uszkoreit, Jones, Gomez, Kaiser, Polosukhin
    Abstract: We propose a new network architecture, the Transformer, based solely
    on attention mechanisms, dispensing with recurrence and convolutions entirely.
    The Transformer allows for significantly more parallelization and achieves
    new state of the art in translation quality. ArXiv ID: 1706.03762""",

    """Title: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
    Authors: Lewis, Perez, Piktus, Petroni, Karpukhin, Goyal, Kuttler, Lewis, Yih, Rocktaschel, Riedel, Kiela
    Abstract: We explore a general-purpose fine-tuning recipe for retrieval-augmented
    generation (RAG) which combines pre-trained parametric and non-parametric memory
    for language generation. ArXiv ID: 2005.11401""",

    """Title: Self-RAG: Learning to Retrieve, Generate, and Critique through Self-Reflection
    Authors: Asai, Wu, Wang, Sil, Hajishirzi
    Abstract: We introduce Self-RAG, a framework that trains a single LM to adaptively
    retrieve passages on demand, generate text informed by retrieved passages, and
    critique its own output using special reflection tokens. Self-RAG significantly
    outperforms existing RAG approaches and LLMs on multiple tasks. ArXiv ID: 2310.11511""",
]

sci_docs = DocumentLoader.from_texts(sample_scientific, "scientific", "demo_papers")
sci_chunks = chunker.chunk_documents(sci_docs, domain="scientific")
count = vs.add_documents("scientific", sci_chunks)
print(f"✅ Indexed {count} scientific chunks")

# Status check
print("\n📊 Index Status:")
for stat in vs.get_all_stats():
    print(f"  {stat['domain']:>12}: {stat['document_count']} chunks")

In [ ]:
import sys

# Make sure the project root is at the FRONT of the path
PROJECT = '/content/drive/MyDrive/AI574_Multi_Domain_Agent/project'
if PROJECT not in sys.path:
    sys.path.insert(0, PROJECT)

# Verify the right modules are found
import agents.industrial_agent
print(f"✅ Found: {agents.industrial_agent.__file__}")

In [ ]:
import importlib, orchestration.workflow_graph, orchestration.state_schema
importlib.reload(orchestration.state_schema)
importlib.reload(orchestration.workflow_graph)
from orchestration.workflow_graph import build_workflow, run_query

try:
    llm, vs, builder
except NameError:
    raise RuntimeError(
        "Run the cells above first so 'llm', 'vs', and 'builder' exist: "
        "load LLM, create embedder & vector store, run IndexBuilder and index data."
    )

workflow = build_workflow(llm=llm, vector_store=vs, index_builder=builder)
print("✅ Workflow compiled!")



In [ ]:
def display_result(result):
    """Pretty-print query result with visual timing breakdown."""
    domain = result.get("domain", "?")
    conf   = result.get("confidence", 0)
    status = result.get("status", "?")
    esc    = result.get("escalated", False)
    srcs   = result.get("sources", [])

    print(f"\n{'='*60}")
    print(f"  Domain: {domain:<14} Confidence: {conf:.2f}")
    print(f"  Status: {status:<14} Escalated:  {esc}")
    print(f"  Sources: {len(srcs)}")
    print(f"{'='*60}")
    print(f"\n{result.get('response', '')}")

    # ── Timing ──────────────────────────────────────────────────
    t = result.get("timing")
    if not t:
        print("\n  (no timing data — run sync cell to update workflow_graph.py)")
        return

    total = t.get("total_s", 0) or 0.001
    sup   = t.get("supervisor_s", 0)
    agent = t.get("agent_s", 0)
    c     = t.get("crag") or {}

    retrieve = c.get("retrieve_s", 0)
    grade    = c.get("grade_s", 0)
    rewrite  = c.get("rewrite_s", 0)
    generate = c.get("generate_s", 0)
    validate = c.get("validate_s", 0)
    overhead = max(0, agent - (retrieve + grade + rewrite + generate + validate))

    steps = [
        ("Supervisor", sup),
        ("  Retrieve", retrieve),
        ("  Grade",    grade),
        ("  Rewrite",  rewrite),
        ("  Generate", generate),
        ("  Validate", validate),
        ("  Overhead", overhead),
    ]

    BAR = 25
    print(f"\n{'─'*60}")
    print(f"  TIMING BREAKDOWN  (total {total:.1f}s / {total/60:.2f} min)")
    print(f"{'─'*60}")
    for label, s in steps:
        if s < 0.01 and label == "  Rewrite":
            continue
        pct  = (s / total) * 100
        fill = int(BAR * s / total)
        bar  = "█" * fill + "░" * (BAR - fill)
        print(f"  {label:<12} {bar} {s:5.1f}s  ({pct:4.1f}%)")
    print(f"{'─'*60}")
    print(f"  {'Total':<12} {'':>{BAR}} {total:5.1f}s")
    print(f"{'─'*60}")

# ── Run test query ──────────────────────────────────────────────────────────
result = run_query(workflow, "My PowerFlex 525 drive is showing fault F004, how do I fix it?")
display_result(result)

In [ ]:
# Test: Recipe query
result = run_query(workflow, "What can I substitute for eggs in cookies?")
display_result(result)

In [ ]:
# Test: Scientific query
result = run_query(workflow, "Summarize the key ideas behind RAG in NLP")
display_result(result)


In [ ]:
# Test: Ambiguous query (should trigger clarification)
result = run_query(workflow, "What temperature should I set to avoid failure?")
display_result(result)

## Why does one query take 3+ minutes? / LLM throughput on A100

**Reasons:**
1. **Multiple LLM calls per query** — Supervisor (route) + batch grader + response generator + hallucination check = **4 sequential calls**. Each call is a full 9B forward/generation.
2. **Keras/JAX is not an inference-optimized stack** — Frameworks like TensorRT-LLM or vLLM get **~50–60 tok/s** for 8–10B on A100. Keras Hub on JAX typically does **~5–20 tok/s** (no fused kernels, no continuous batching).
3. **Rough token budget per query** — ~80 (supervisor) + ~300 (grader) + ~400–800 (generator) + ~80 (hallucination) ≈ **900–1300 output tokens**. At 10 tok/s that’s 90–130 s just for generation; add overhead and you get 3+ min.

**To see your actual throughput:** run the cell below and check the log lines `LLM call: Xs | ~Y output tokens | ~Z tok/s`.

In [ ]:
# Enable INFO logging so "LLM call: ... tok/s" lines appear
import logging
logging.basicConfig(level=logging.INFO, format="%(message)s")
for _ in ["foundation.llm_wrapper", "rag_core.crag_pipeline"]:
    logging.getLogger(_).setLevel(logging.INFO)

# Timed re-run with full breakdown
result = run_query(workflow, "My PowerFlex 525 drive is showing fault F004, how do I fix it?")
display_result(result)

# New Section